[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vrra/FGAN-Build-a-thon-2022/blob/main/Notebooks2022/build_a_thon_graph_v1.ipynb)

## Build-a-thon 2022, Hoist the colors?
v1.0 release on 24 Aug 2022
Metrics: 325 nodes, 592 relations

In [ ]:
!rm -rf /tmp/FGAN-Build-a-thon-2022/
!git clone https://github.com/vrra/FGAN-Build-a-thon-2022.git /tmp/FGAN-Build-a-thon-2022


In [1]:
from IPython.display import Image
Image(filename='/tmp/FGAN-Build-a-thon-2022/Banner/1920x1080-FG_AN-buildathon_Sep02-01.png', width=500) 


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/FGAN-Build-a-thon-2022/Banner/1920x1080-FG_AN-buildathon_Sep02-01.png'

In [ ]:
!rm -rf /tmp/FGAN-Build-a-thon-2022/

## Introduction

Created: 14 Apr 2022.   
Author: Vishnu Ram OV.   
vishnu.n@ieee.org.   
Licence: available for all purposes under the sun  
but with acknowledgement and citation to "ITU FG AN Build-a-thon 2022"


## References 
[1] Neo4j Docs, "Creating an instance", https://neo4j.com/docs/aura/auradb/getting-started/create-database/   
[2] ITU-T Focus Group Autonomous Networks, "Use cases for Autonomous Networks", https://extranet.itu.int/sites/itu-t/focusgroups/an/output/FGAN-O-013-R1.docx  
[3] Build-a-thon github, https://github.com/vrra/FGAN-Build-a-thon-2022

## Prerequisite: 
create a free account at https://console.neo4j.io  
Follow steps at https://extranet.itu.int/sites/itu-t/focusgroups/an/input/FGAN-I-253.pdf  


## General cautions
Property names have strict conventions and rules in neo4j (apparently)

### For Colab users:

If you are using colab, add your credentials here and uncomment these  
and skip to step-1 below.

In [ ]:
#uri = "your uri from console.neo4j.io"  
#user = "your username from console.neo4j.io"  
#password = "your passwd from console.neo4j.io"  

### For non-colab users:
Create a local file .env with   
neo4j_uri="your uri from console.neo4j.io"  
neo4j_user="your username from console.neo4j.io"  
neo4j_password="your passwd from console.neo4j.io"  

#### Read the neo4j URI, username, passwd


In [2]:
import os
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [3]:

#Vishnu: 1 Jun 2022: use the details from https://console.neo4j.io
#especially use your own DB instance and corresponding passwd.
#if you dont have a .env file, replace here with your credentials.

uri = os.getenv('neo4j_uri')
user = os.getenv('neo4j_user')
password = os.getenv('neo4j_password')

## step-1: install neo4j

In [4]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.


## step-2: imports and init

In [5]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

## step-3: define helper class

In [37]:


class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    #Vishnu: 1 Aug 2022: uses labels instead of properties
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #        (Note that rel still uses properties and not labels)
    def create_node_with_usecase_label(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node_label, actor_name)
            if not node_already_exists:  
                #node itself doesnt exist (not to mention the use case label), Lets add.
                result = session.write_transaction(
                    self._create_and_return_node_label, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'], 
                                                                 usecase=row['n1usecase']))
                    if not (usecase_id in row['n1usecase']):
                      print("adding label "
                                +usecase_id+ 
                                " in addition to {usecase}".format(usecase=row['n1usecase']))
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node_label, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break
    
    #Vishnu: 1 Aug 2022: created this f() to query nodes based on a name
    #                    and return with usecase labels
    @staticmethod
    def _find_and_return_existing_node_label(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, labels(n1) as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to create nodes with usecase label
    @staticmethod
    def _create_and_return_node_label(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 :" + usecase_id + " { name: '" + actor_name + "'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: created this f() to add use case labels to existing nodes
    #                    Note that the existing labels are left there.
    @staticmethod
    def _add_usecase_and_return_existing_node_label(tx, actor_name, usecase_id):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1 :" + usecase_id +" "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to return all use case actors based on labels
    #                    as against properties
    @staticmethod
    def _find_and_return_all_usecase_actors_label(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE '" +usecase_id+ "' in labels(n) "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 1 Aug 2022: created this f() to list all nodes with usecase label
    #                    instead of properties
    def find_all_usecase_actors_label(self, usecase_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors_label, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    #Vishnu: 1 Aug 2022: Created to use label for use case.
    #CAUTION: Assumes that actors already exists with label as use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase_label(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case, 
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel_label, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'], 
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))

    #Vishnu: 1 Aug 2022 
    #assumption: nodes already exist
    #modified from _create_and_return_rel to use labels for n.usecase
    @staticmethod
    def _create_and_return_rel_label(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND '"+ usecaseid+"' in labels(n1) "
            " AND '"+ usecaseid+"' in labels(n2) "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"], 
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for n.usecases.
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #Vishnu: 22 Jun 2022: refactored, uses properties instead of labels
    #Vishnu: 1 Jun 2022: created this f() to create nodes with usecase label
    #Vishnu: 2 Jun 2022: added- check if it already exists.
    #Vishnu: 2 Jun 2022: added- check the usecase duplication, adds a usecase if it doesnt exist.
    #otherwise ignores the command.
    def create_node_with_usecase(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node, actor_name)
            if not node_already_exists:  
                result = session.write_transaction(
                    self._create_and_return_node, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'], 
                                                                 usecase=row['n1usecase']))
                    s=row['n1usecase'].split(',')
                    if not (usecase_id in s):
                      print("adding prop "+usecase_id+ " in addition to "+row['n1usecase'])
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    #Vishnu: 2 Jun 2022: created this f() to query nodes and return with usecase prop
    @staticmethod
    def _find_and_return_existing_node(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, n1.usecase as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _create_and_return_node(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 "+"{ name: '"+actor_name+"', usecase: '"+usecase_id+"'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    #Vishnu: 1 Jun 2022: created this f() to add use case prop to existing nodes 
    #CAUTION: does not check for duplication, use create_node_with_prop instead.
    def add_usecase_existing_node(self, actor_name, usecase_id):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._add_usecase_and_return_existing_node, actor_name, usecase_id)
            
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added usecase to node: {n1}".format(n1=row['n1']))

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _add_usecase_and_return_existing_node(tx, actor_name, usecase_id):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1.usecase = n1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: Deprecated: This approach uses properties instead of labels for usecases.
    #CAUTION: Assumes that actors already exists with property as n.use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case, 
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'], 
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))
    
    #Vishnu: 1 Aug 2022: at this moment this f() uses labels instead of properties for n.usecases.
    #                    Note that rels still uses properties.
    #        1 Aug 2022: TBD: create a property-version (as against label) of the same thing.
    #Vishnu: 21 Jun 2022: created this f() to create relationships with usecase label
    #if this works, this is the only function that needed to be called.
    def create_actors_relationship_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
            #process actor1
            self.create_node_with_usecase_label(actor1_name, usecase_id)
            #process actor2
            self.create_node_with_usecase_label(actor2_name, usecase_id)
            #process relation
            self.create_rel_with_usecase_label(actor1_name, rel_name, actor2_name, usecase_id)

    #Vishnu: 23 June 2022: created
    #looks for a rel under a use case between 2 given nodes.
    #normally, we expect only 0 or 1 unique entry.
    @staticmethod                
    def _find_and_return_this_existing_rels(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "and   (r1.name ='" + rel_name + "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        return [row["r1name"] for row in result]
        
    
    #Vishnu: 21 June 2022: created this supporting f()
    #return for all relationship between given two nodes.
    @staticmethod                
    def _find_and_return_all_existing_rels_for_uc(tx, actor1_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        try:
            return [{
                        "n1name": row["n1name"],
                        "n2name": row["n2name"],
                        "r1name": row["r1name"],
                        "r1usecase": row["r1usecase"],
                    } 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 21 June 2022: created this supporting f()
    #adds a name for an existing relationship between given two nodes.
    @staticmethod                
    def _add_usecase_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "')"
            "set    r1.usecase = r1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
        )
        result = tx.run(query)

    
    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists, 
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_node(tx, actor_name, propertyName, propertyVal):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1."+propertyName+" = '"+propertyVal +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists, 
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name, 
                                                propertyName, propertyVal):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.name ='" + rel_name + "') "
            "set   r1."+propertyName+" = '"+propertyVal +"' "
            "return r1"
        )
        result = tx.run(query)
        try:
            return [{"r1": row["r1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 22 June 2022: created
    #assumption: nodes already exist
    @staticmethod
    def _create_and_return_rel(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND n1.usecase CONTAINS '"+ usecaseid+"' "
            " AND n2.usecase CONTAINS '"+ usecaseid+"' "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"], 
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Jun 2022: created this f() to list all nodes with usecase label
    def find_all_usecase_actors(self, usecase_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    @staticmethod
    def _find_and_return_all_usecase_actors(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE (n.usecase = '"+usecase_id+"') "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 21 Jun 2022: created this f() to cleanup the DB
    #CAUTION: will delete everything
    def cleanup_db(self):
        with self.driver.session() as session:
            result = session.read_transaction(
                self._count_nodes_in_db)
            for row in result:
                print("Found {row} actors, deleting! ".format(row=row['count']))
            result = session.write_transaction(
                self._cleanup_db)

    @staticmethod
    def _count_nodes_in_db(tx):
        query = (
            "MATCH (n) "
            "return count(n) as count"
        )
        result = tx.run(query)
        try:
            return [{"count": row["count"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 22 June 2022: created this f() 
    #(over)writes a prop to a node
    def writePropToNode(self, actor_name, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_node, 
                    actor_name, 
                    propertyName, 
                    propertyVal)
        
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to node: {n1}".format(n1=row['n1']))

    #Vishnu: 22 June 2022: created this f() 
    #(over)writes a prop to a rel
    def writePropToRel(self, a1, rel_name, a2, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_rel, 
                    a1, rel_name, a2,
                    propertyName, 
                    propertyVal)
        
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to rel: {r1}".format(r1=row['r1']))

    @staticmethod
    def _cleanup_db(tx):
        query = (
            "MATCH (n) "
            "detach delete n"
        )
        result = tx.run(query)
        try:
            return 
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
    
    #Vishnu: 29 Aug 2022
    #https://github.com/vrra/FGAN-Build-a-thon-2022/issues/52
    def del_node_in_db(self, node_name):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node_label, node_name)
            if not node_already_exists:
                print(f"\"{node_name}\" does not exist! ")
            result = session.write_transaction(
                self._del_node_in_db, node_name)
            for row in result:
                    print("Deleted node: {n1}".format(n1=row['name']))
            
    #Vishnu: 29 Aug 2022
    #https://github.com/vrra/FGAN-Build-a-thon-2022/issues/52
    @staticmethod
    def _del_node_in_db(tx, actor1_name):
        query = (
            "MATCH (n) "
            "WHERE (n.name = '"+actor1_name+"') "
            "with n, n.name as name "
            "detach delete n "
            "return name"
        )
        result = tx.run(query)
        try:
            return [{"name": row["name"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

## step-4: instantiate the helper class to DB. 
CAUTION: this uses the username/passwd assigned already in step-2 above.

In [38]:
app = App(uri, user, password)

## step-5 clean up DB
#Caution: lose data


In [41]:
app.cleanup_db()

Found 331 actors, deleting! 


## Examples

In [40]:
app.del_node_in_db("traffic offload controller")
app.del_node_in_db("supply chain management controller")
app.del_node_in_db("NF configure request")


"traffic offload controller" does not exist! 
"supply chain management controller" does not exist! 
"NF configure request" does not exist! 


In [ ]:
app.create_actors_relationship_with_usecase("Experimentation Controller", 
                                            "myrel", 
                                            "Sandbox",
                                            "usecase_001")

### Example-1: Create individual nodes with usecase
(and add a node property)
(and add a rel property)

In [ ]:
app.create_node_with_usecase("Experimentation Controller", "usecase_001")
app.create_node_with_usecase("Sandbox", "usecase_001")


In [ ]:
app.writePropToNode("Sandbox", "provide", "import knowledge")


In [ ]:
app.writePropToNode("Sandbox", "need", "integrate controllers")


### Example-1a: try to create duplicate node under different use case

In [ ]:
app.create_node_with_usecase("Sandbox", "usecase_003")
app.create_node_with_usecase("Sandbox", "usecase_004")
app.create_node_with_usecase("Sandbox", "usecase_005")
app.create_node_with_usecase("Sandbox", "usecase_006")


### Example-1b: use create_node_with_usecase_label (instead of properties)

In [ ]:
app.create_node_with_usecase_label("Sandbox", "usecase_003")
app.create_node_with_usecase_label("Sandbox", "usecase_003")
app.create_node_with_usecase_label("Sandbox", "usecase_004")
app.create_node_with_usecase_label("Sandbox", "usecase_004")
app.create_node_with_usecase_label("Sandbox", "usecase_005")



### Example-2: Create relation between 2 existing nodes

In [ ]:
app.create_rel_with_usecase("Experimentation Controller", "my_best_buddy", "Sandbox", "usecase_001")
app.create_rel_with_usecase("Experimentation Controller", "my_best_buddy", "Sandbox", "usecase_002")
app.create_rel_with_usecase("Experimentation Controller", "my_best_buddy", "Sandbox", "usecase_003")


In [ ]:
app.writePropToRel("Experimentation Controller", "my_best_buddy", "Sandbox", 
                    "my_rel_prop_name", 
                    "my-rel-prop-val")

## Cleanup DB 
CAUTION: Remember to clean up the DB before executing the next steps. (so that all examples above are not leftover in the use case data below).

In [ ]:
app.cleanup_db()

## USECASES

### FG-AN-USECASE-001

In [42]:

app.create_actors_relationship_with_usecase("Knowledge Base", "import", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("Knowledge Base", "export", "Knowledge Base", "usecase_001")

app.create_actors_relationship_with_usecase("Knowledge Base Manager", "optimizes", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("Knowledge Base Manager", "export", "Knowledge Base", "usecase_001")

app.create_actors_relationship_with_usecase("AN Orchestrator", "refer", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("AN Orchestrator", "input", "Use case description", "usecase_001")
app.create_actors_relationship_with_usecase("AN Orchestrator", "output", "TOSCA format", "usecase_001")

app.create_actors_relationship_with_usecase("Auto controller generator", "refer", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("Auto controller generator", "output", "TOSCA format", "usecase_001")

app.create_actors_relationship_with_usecase("controller repository", "stores", "controllers", "usecase_001")
app.create_actors_relationship_with_usecase("ML model", "recommends", "controllers", "usecase_001")

app.create_actors_relationship_with_usecase("Human operator", "input", "reports", "usecase_001")
app.create_actors_relationship_with_usecase("Human operator", "monitors", "reports", "usecase_001")

Created node: Knowledge Base
Node already exists: Knowledge Base in ['usecase_001']
ignoring ...
Created relation: Knowledge Base - import - Knowledge Base in usecase_001
Node already exists: Knowledge Base in ['usecase_001']
ignoring ...
Node already exists: Knowledge Base in ['usecase_001']
ignoring ...
Created relation: Knowledge Base - export - Knowledge Base in usecase_001
Created node: Knowledge Base Manager
Node already exists: Knowledge Base in ['usecase_001']
ignoring ...
Created relation: Knowledge Base Manager - optimizes - Knowledge Base in usecase_001
Node already exists: Knowledge Base Manager in ['usecase_001']
ignoring ...
Node already exists: Knowledge Base in ['usecase_001']
ignoring ...
Created relation: Knowledge Base Manager - export - Knowledge Base in usecase_001
Created node: AN Orchestrator
Node already exists: Knowledge Base in ['usecase_001']
ignoring ...
Created relation: AN Orchestrator - refer - Knowledge Base in usecase_001
Node already exists: AN Orchest

In [43]:
app.find_all_usecase_actors_label("usecase_001")

Found actor-1: TOSCA format
Found actor-2: Auto controller generator
Found actor-3: controller repository
Found actor-4: controllers
Found actor-5: ML model
Found actor-6: Human operator
Found actor-7: reports
Found actor-8: AN Orchestrator
Found actor-9: Use case description
Found actor-10: Knowledge Base Manager
Found actor-11: Knowledge Base


### FG-AN-USECASE-002


In [44]:
app.create_actors_relationship_with_usecase("Experimentation controller", "decides", 
                                        "scenarios for experimentation", "usecase_002")
app.create_actors_relationship_with_usecase("AN sandbox", "hosts", 
                                        "simulators", "usecase_002")
app.create_actors_relationship_with_usecase("AN sandbox", "manages", 
                                        "simulators", "usecase_002")
app.create_actors_relationship_with_usecase("AN sandbox", "output", 
                                        "reports", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "triggers", 
                                        "experiments", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "output", 
                                        "reports", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "executes", 
                                        "experiments", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "monitors", 
                                        "experiments", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "uses", 
                                        "AN sandbox", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "updates", 
                                        "Knowledge Base", "usecase_002")
app.create_actors_relationship_with_usecase("experiments", "has", 
                                        "scenarios for experimentation", "usecase_002")

Created node: Experimentation controller
Created node: scenarios for experimentation
Created relation: Experimentation controller - decides - scenarios for experimentation in usecase_002
Created node: AN sandbox
Created node: simulators
Created relation: AN sandbox - hosts - simulators in usecase_002
Node already exists: AN sandbox in ['usecase_002']
ignoring ...
Node already exists: simulators in ['usecase_002']
ignoring ...
Created relation: AN sandbox - manages - simulators in usecase_002
Node already exists: AN sandbox in ['usecase_002']
ignoring ...
Node already exists: reports in ['usecase_001']
adding label usecase_002 in addition to ['usecase_001']
Created relation: AN sandbox - output - reports in usecase_002
Node already exists: Experimentation controller in ['usecase_002']
ignoring ...
Created node: experiments
Created relation: Experimentation controller - triggers - experiments in usecase_002
Node already exists: Experimentation controller in ['usecase_002']
ignoring ...
N

In [45]:
app.find_all_usecase_actors_label("usecase_002")

Found actor-1: reports
Found actor-2: Experimentation controller
Found actor-3: scenarios for experimentation
Found actor-4: AN sandbox
Found actor-5: simulators
Found actor-6: experiments
Found actor-7: Knowledge Base


### FG-AN-USECASE-003


In [46]:
app.create_actors_relationship_with_usecase("AN Orchestrator", "queries", 
                                        "AN Orchestrator", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "queries", 
                                        "Human operator", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "accept", 
                                        "feedback", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "configures", 
                                        "AN components", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "updates", 
                                        "Knowledge Base", "usecase_003")


Node already exists: AN Orchestrator in ['usecase_001']
adding label usecase_003 in addition to ['usecase_001']
Node already exists: AN Orchestrator in ['usecase_001', 'usecase_003']
ignoring ...
Created relation: AN Orchestrator - queries - AN Orchestrator in usecase_003
Node already exists: AN Orchestrator in ['usecase_001', 'usecase_003']
ignoring ...
Node already exists: Human operator in ['usecase_001']
adding label usecase_003 in addition to ['usecase_001']
Created relation: AN Orchestrator - queries - Human operator in usecase_003
Node already exists: AN Orchestrator in ['usecase_001', 'usecase_003']
ignoring ...
Created node: feedback
Created relation: AN Orchestrator - accept - feedback in usecase_003
Node already exists: AN Orchestrator in ['usecase_001', 'usecase_003']
ignoring ...
Created node: AN components
Created relation: AN Orchestrator - configures - AN components in usecase_003
Node already exists: AN Orchestrator in ['usecase_001', 'usecase_003']
ignoring ...
Node a

In [47]:
app.find_all_usecase_actors_label("usecase_003")

Found actor-1: Human operator
Found actor-2: feedback
Found actor-3: AN components
Found actor-4: AN Orchestrator
Found actor-5: Knowledge Base


### FG-AN-USECASE-004

In [48]:
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", 
                                        "controllers", "usecase_004")
app.create_actors_relationship_with_usecase("Adaptation controller", "deploy", 
                                        "controllers", "usecase_004")
app.create_actors_relationship_with_usecase("Adaptation controller", "configures", 
                                        "controllers", "usecase_004")
app.create_actors_relationship_with_usecase("Adaptation controller", "queries", 
                                        "controller repository", "usecase_004")          
app.create_actors_relationship_with_usecase("Adaptation controller", "updates", 
                                        "Knowledge Base", "usecase_004")  
app.create_actors_relationship_with_usecase("controllers", "is", 
                                        "undeployed controllers", "usecase_004")  
app.create_actors_relationship_with_usecase("controllers", "is", 
                                        "deployed controllers", "usecase_004")                                         

Created node: Adaptation controller
Node already exists: controllers in ['usecase_001']
adding label usecase_004 in addition to ['usecase_001']
Created relation: Adaptation controller - discovers - controllers in usecase_004
Node already exists: Adaptation controller in ['usecase_004']
ignoring ...
Node already exists: controllers in ['usecase_001', 'usecase_004']
ignoring ...
Created relation: Adaptation controller - deploy - controllers in usecase_004
Node already exists: Adaptation controller in ['usecase_004']
ignoring ...
Node already exists: controllers in ['usecase_001', 'usecase_004']
ignoring ...
Created relation: Adaptation controller - configures - controllers in usecase_004
Node already exists: Adaptation controller in ['usecase_004']
ignoring ...
Node already exists: controller repository in ['usecase_001']
adding label usecase_004 in addition to ['usecase_001']
Created relation: Adaptation controller - queries - controller repository in usecase_004
Node already exists: Ad

In [49]:
app.find_all_usecase_actors_label("usecase_004")

Found actor-1: controller repository
Found actor-2: controllers
Found actor-3: Adaptation controller
Found actor-4: undeployed controllers
Found actor-5: deployed controllers
Found actor-6: Knowledge Base


### FG-AN-USECASE-005

In [50]:
app.create_actors_relationship_with_usecase("Service management framework", "is", 
                                        "underlay", "usecase_005")
app.create_actors_relationship_with_usecase("Adaptation controller", "discover", 
                                        "Service management framework", "usecase_005")     
app.create_actors_relationship_with_usecase("Adaptation controller", "influence", 
                                        "Service management framework", "usecase_005")  
app.create_actors_relationship_with_usecase("intent", "is", 
                                        "influence", "usecase_005")  
app.create_actors_relationship_with_usecase("policy", "is", 
                                        "influence", "usecase_005")                                         

Created node: Service management framework
Created node: underlay
Created relation: Service management framework - is - underlay in usecase_005
Node already exists: Adaptation controller in ['usecase_004']
adding label usecase_005 in addition to ['usecase_004']
Node already exists: Service management framework in ['usecase_005']
ignoring ...
Created relation: Adaptation controller - discover - Service management framework in usecase_005
Node already exists: Adaptation controller in ['usecase_004', 'usecase_005']
ignoring ...
Node already exists: Service management framework in ['usecase_005']
ignoring ...
Created relation: Adaptation controller - influence - Service management framework in usecase_005
Created node: intent
Created node: influence
Created relation: intent - is - influence in usecase_005
Created node: policy
Node already exists: influence in ['usecase_005']
ignoring ...
Created relation: policy - is - influence in usecase_005


In [51]:
app.find_all_usecase_actors_label("usecase_005")

Found actor-1: Adaptation controller
Found actor-2: Service management framework
Found actor-3: underlay
Found actor-4: intent
Found actor-5: influence
Found actor-6: policy


### FG-AN-USECASE-006 cat1

In [52]:
app.create_actors_relationship_with_usecase("Evolution controller", "monitors", 
                                        "controllers", "usecase_006_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "discovers", 
                                        "use cases", "usecase_006_cat1")     
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", 
                                        "controllers", "usecase_006_cat1")  
app.create_actors_relationship_with_usecase("Evolution controller", "refers", 
                                        "Knowledge Base", "usecase_006_cat1")  

Created node: Evolution controller
Node already exists: controllers in ['usecase_001', 'usecase_004']
adding label usecase_006_cat1 in addition to ['usecase_001', 'usecase_004']
Created relation: Evolution controller - monitors - controllers in usecase_006_cat1
Node already exists: Evolution controller in ['usecase_006_cat1']
ignoring ...
Created node: use cases
Created relation: Evolution controller - discovers - use cases in usecase_006_cat1
Node already exists: Evolution controller in ['usecase_006_cat1']
ignoring ...
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1']
ignoring ...
Created relation: Evolution controller - recommends - controllers in usecase_006_cat1
Node already exists: Evolution controller in ['usecase_006_cat1']
ignoring ...
Node already exists: Knowledge Base in ['usecase_001', 'usecase_002', 'usecase_003', 'usecase_004']
adding label usecase_006_cat1 in addition to ['usecase_001', 'usecase_002', 'usecase_003', 'usecase_004']
Cr

In [53]:
app.find_all_usecase_actors_label("usecase_006_cat1")

Found actor-1: controllers
Found actor-2: Evolution controller
Found actor-3: use cases
Found actor-4: Knowledge Base


### FG-AN-USECASE-006 cat2

In [54]:
app.create_actors_relationship_with_usecase("Data collector", "consumes", 
                                        "data", "usecase_006_cat2")
app.create_actors_relationship_with_usecase("Management domain", "produces", 
                                        "data", "usecase_006_cat2")     
app.create_actors_relationship_with_usecase("Management domain", "has", 
                                        "RAN cell", "usecase_006_cat2")  
app.create_actors_relationship_with_usecase("Data collector", "produces", 
                                        "data", "usecase_006_cat2") 

app.create_actors_relationship_with_usecase("data", "has", 
                                        "real time", "usecase_006_cat2")     
app.create_actors_relationship_with_usecase("data", "has", 
                                        "data quality", "usecase_006_cat2")  
app.create_actors_relationship_with_usecase("data", "has", 
                                        "KPI", "usecase_006_cat2") 

app.create_actors_relationship_with_usecase("Data monitor", "consumes", 
                                        "data", "usecase_006_cat2")     
app.create_actors_relationship_with_usecase("Data monitor", "has", 
                                        "data monitoring", "usecase_006_cat2")  
app.create_actors_relationship_with_usecase("Data monitor", "produces", 
                                        "reports", "usecase_006_cat2") 
app.create_actors_relationship_with_usecase("reports", "has", 
                                        "visualizations", "usecase_006_cat2")                                         
app.create_actors_relationship_with_usecase("Data analytics", "consumes", 
                                        "data", "usecase_006_cat2")    
app.create_actors_relationship_with_usecase("Data analytics", "produces", 
                                        "inference", "usecase_006_cat2")                                           
app.create_actors_relationship_with_usecase("Data analytics", "has", 
                                        "ML model", "usecase_006_cat2") 
app.create_actors_relationship_with_usecase("inference", "has", 
                                        "periodicity", "usecase_006_cat2")      
app.create_actors_relationship_with_usecase("ML model", "has", 
                                        "model training", "usecase_006_cat2")

Created node: Data collector
Created node: data
Created relation: Data collector - consumes - data in usecase_006_cat2
Created node: Management domain
Node already exists: data in ['usecase_006_cat2']
ignoring ...
Created relation: Management domain - produces - data in usecase_006_cat2
Node already exists: Management domain in ['usecase_006_cat2']
ignoring ...
Created node: RAN cell
Created relation: Management domain - has - RAN cell in usecase_006_cat2
Node already exists: Data collector in ['usecase_006_cat2']
ignoring ...
Node already exists: data in ['usecase_006_cat2']
ignoring ...
Created relation: Data collector - produces - data in usecase_006_cat2
Node already exists: data in ['usecase_006_cat2']
ignoring ...
Created node: real time
Created relation: data - has - real time in usecase_006_cat2
Node already exists: data in ['usecase_006_cat2']
ignoring ...
Created node: data quality
Created relation: data - has - data quality in usecase_006_cat2
Node already exists: data in ['

In [55]:
app.find_all_usecase_actors_label("usecase_006_cat2")

Found actor-1: ML model
Found actor-2: reports
Found actor-3: Data collector
Found actor-4: data
Found actor-5: Management domain
Found actor-6: RAN cell
Found actor-7: real time
Found actor-8: data quality
Found actor-9: KPI
Found actor-10: Data monitor
Found actor-11: data monitoring
Found actor-12: visualizations
Found actor-13: Data analytics
Found actor-14: inference
Found actor-15: periodicity
Found actor-16: model training


### FG-AN-USECASE-007 cat1

In [56]:
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers deployed", 
                                        "CL automation framework", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "deploys", 
                                        "controllers", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "automatic data acquisition", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "ML model", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "policy", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "action implementation", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("policy", "need", 
                                        "optimization", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "configures", 
                                        "controllers", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", 
                                        "controllers", "usecase_007_cat1")                                       
app.create_actors_relationship_with_usecase("Adaptation controller", "adapts", 
                                        "controllers", "usecase_007_cat1") 
app.create_actors_relationship_with_usecase("Adaptation controller", "interfaces with", 
                                        "underlay orchestrator", "usecase_007_cat1")   

Node already exists: Adaptation controller in ['usecase_004', 'usecase_005']
adding label usecase_007_cat1 in addition to ['usecase_004', 'usecase_005']
Created node: CL automation framework
Created relation: Adaptation controller - discovers deployed - CL automation framework in usecase_007_cat1
Node already exists: CL automation framework in ['usecase_007_cat1']
ignoring ...
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1']
adding label usecase_007_cat1 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1']
Created relation: CL automation framework - deploys - controllers in usecase_007_cat1
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1']
ignoring ...
Created node: automatic data acquisition
Created relation: controllers - has - automatic data acquisition in usecase_007_cat1
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1'

In [57]:
app.find_all_usecase_actors_label("usecase_007_cat1")

Found actor-1: controllers
Found actor-2: ML model
Found actor-3: Adaptation controller
Found actor-4: policy
Found actor-5: CL automation framework
Found actor-6: automatic data acquisition
Found actor-7: action implementation
Found actor-8: optimization
Found actor-9: underlay orchestrator


### FG-AN-USECASE-007 cat2

In [58]:
app.create_actors_relationship_with_usecase("Energy saving controller", "uses", 
                                        "modules", "usecase_007_cat2")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "modules", "usecase_007_cat2")    
app.create_actors_relationship_with_usecase("data acquisition", "is", 
                                        "modules", "usecase_007_cat2")
app.create_actors_relationship_with_usecase("policy", "is", 
                                        "modules", "usecase_007_cat2")  
app.create_actors_relationship_with_usecase("Action implementation", "is", 
                                        "modules", "usecase_007_cat2")        
app.create_actors_relationship_with_usecase("Data acquisition", "uses", 
                                        "data center APIs", "usecase_007_cat2")   
app.create_actors_relationship_with_usecase("policy", "has", 
                                        "data center policy", "usecase_007_cat2")   
app.create_actors_relationship_with_usecase("Action implementation", "uses", 
                                        "data center APIs", "usecase_007_cat2")        
app.create_actors_relationship_with_usecase("controllers", "need", 
                                        "evaluation", "usecase_007_cat2")                
app.create_actors_relationship_with_usecase("evaluation", "has", 
                                        "metrics", "usecase_007_cat2")        
app.create_actors_relationship_with_usecase("metrics", "are", 
                                        "data center metrics", "usecase_007_cat2") 
app.create_actors_relationship_with_usecase("controllers", "need", 
                                        "optimization", "usecase_007_cat2")     
app.create_actors_relationship_with_usecase("optimization", "uses", 
                                        "evaluation", "usecase_007_cat2")                                                

Created node: Energy saving controller
Created node: modules
Created relation: Energy saving controller - uses - modules in usecase_007_cat2
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1']
adding label usecase_007_cat2 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1']
Node already exists: modules in ['usecase_007_cat2']
ignoring ...
Created relation: controllers - has - modules in usecase_007_cat2
Created node: data acquisition
Node already exists: modules in ['usecase_007_cat2']
ignoring ...
Created relation: data acquisition - is - modules in usecase_007_cat2
Node already exists: policy in ['usecase_005', 'usecase_007_cat1']
adding label usecase_007_cat2 in addition to ['usecase_005', 'usecase_007_cat1']
Node already exists: modules in ['usecase_007_cat2']
ignoring ...
Created relation: policy - is - modules in usecase_007_cat2
Created node: Action implementation
Node already exists: module

In [59]:
app.find_all_usecase_actors_label("usecase_007_cat2")

Found actor-1: controllers
Found actor-2: policy
Found actor-3: optimization
Found actor-4: Energy saving controller
Found actor-5: modules
Found actor-6: data acquisition
Found actor-7: Action implementation
Found actor-8: Data acquisition
Found actor-9: data center APIs
Found actor-10: data center policy
Found actor-11: evaluation
Found actor-12: metrics
Found actor-13: data center metrics


### FG-AN-USECASE-008 cat1

In [60]:
app.create_actors_relationship_with_usecase("CL automation framework", "discovers", "network parameters", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "discovers", "data", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "discovers", "KPI", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "deploys", "controllers", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("controllers", "provide", "optimization", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("KPI", "need", "optimization", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("use cases", "has", "KPI", "usecase_008_cat1")
          

Node already exists: CL automation framework in ['usecase_007_cat1']
adding label usecase_008_cat1 in addition to ['usecase_007_cat1']
Created node: network parameters
Created relation: CL automation framework - discovers - network parameters in usecase_008_cat1
Node already exists: CL automation framework in ['usecase_007_cat1', 'usecase_008_cat1']
ignoring ...
Node already exists: data in ['usecase_006_cat2']
adding label usecase_008_cat1 in addition to ['usecase_006_cat2']
Created relation: CL automation framework - discovers - data in usecase_008_cat1
Node already exists: CL automation framework in ['usecase_007_cat1', 'usecase_008_cat1']
ignoring ...
Node already exists: KPI in ['usecase_006_cat2']
adding label usecase_008_cat1 in addition to ['usecase_006_cat2']
Created relation: CL automation framework - discovers - KPI in usecase_008_cat1
Node already exists: CL automation framework in ['usecase_007_cat1', 'usecase_008_cat1']
ignoring ...
Node already exists: controllers in ['u

In [61]:
app.create_actors_relationship_with_usecase("Evolution controller", "discovers", "use cases", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "composes", "controllers", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("use cases", "has", "data", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("data", "has", "parameters", "usecase_008_cat1")


Node already exists: Evolution controller in ['usecase_006_cat1']
adding label usecase_008_cat1 in addition to ['usecase_006_cat1']
Node already exists: use cases in ['usecase_006_cat1', 'usecase_008_cat1']
ignoring ...
Created relation: Evolution controller - discovers - use cases in usecase_008_cat1
Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1']
ignoring ...
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1']
ignoring ...
Created relation: Evolution controller - composes - controllers in usecase_008_cat1
Node already exists: use cases in ['usecase_006_cat1', 'usecase_008_cat1']
ignoring ...
Node already exists: data in ['usecase_006_cat2', 'usecase_008_cat1']
ignoring ...
Created relation: use cases - has - data in usecase_008_cat1
Node already exists: data in ['usecase_006_cat2', 'usecase_008_cat1']
ignoring ...
Created node: parameters
Created relati

In [62]:
app.find_all_usecase_actors_label("usecase_008_cat1")

Found actor-1: controllers
Found actor-2: Evolution controller
Found actor-3: use cases
Found actor-4: data
Found actor-5: KPI
Found actor-6: CL automation framework
Found actor-7: optimization
Found actor-8: network parameters
Found actor-9: parameters


### FG-AN-USECASE-008 cat2

In [63]:
app.create_actors_relationship_with_usecase("base station", "has", "UE location", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("mMiMO controller", "needs", "UE location", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("mMiMO controller", "has", "optimization", "usecase_008_cat2") 
app.create_actors_relationship_with_usecase("optimization", "has", "goals", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "system utilization", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "user experience", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "target RSRP/SINR distribution", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "optimal antenna parameters", "usecase_008_cat2")

Created node: base station
Created node: UE location
Created relation: base station - has - UE location in usecase_008_cat2
Created node: mMiMO controller
Node already exists: UE location in ['usecase_008_cat2']
ignoring ...
Created relation: mMiMO controller - needs - UE location in usecase_008_cat2
Node already exists: mMiMO controller in ['usecase_008_cat2']
ignoring ...
Node already exists: optimization in ['usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1']
adding label usecase_008_cat2 in addition to ['usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1']
Created relation: mMiMO controller - has - optimization in usecase_008_cat2
Node already exists: optimization in ['usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_008_cat2']
ignoring ...
Created node: goals
Created relation: optimization - has - goals in usecase_008_cat2
Node already exists: goals in ['usecase_008_cat2']
ignoring ...
Created node: system utilization
Created relation: goals - are 

In [64]:
app.find_all_usecase_actors_label("usecase_008_cat2")

Found actor-1: optimization
Found actor-2: base station
Found actor-3: UE location
Found actor-4: mMiMO controller
Found actor-5: goals
Found actor-6: system utilization
Found actor-7: user experience
Found actor-8: target RSRP/SINR distribution
Found actor-9: optimal antenna parameters


### FG-AN-USECASE-009 cat1

In [65]:
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "controller characteristics", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "accepts", "AN intent", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "composes", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "evolves", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "evolves", "modules", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "implements", "evolution algorithms", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "modules", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "composes", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "deploys", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("controllers", "utilizes", "underlay specific APIs", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("edge controllers", "are", "controllers", "usecase_009_cat1")


Node already exists: Adaptation controller in ['usecase_004', 'usecase_005', 'usecase_007_cat1']
adding label usecase_009_cat1 in addition to ['usecase_004', 'usecase_005', 'usecase_007_cat1']
Created node: controller characteristics
Created relation: Adaptation controller - discovers - controller characteristics in usecase_009_cat1
Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1']
adding label usecase_009_cat1 in addition to ['usecase_006_cat1', 'usecase_008_cat1']
Created node: AN intent
Created relation: Evolution controller - accepts - AN intent in usecase_009_cat1
Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1']
ignoring ...
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1']
adding label usecase_009_cat1 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'useca

In [66]:
app.find_all_usecase_actors_label("usecase_009_cat1")

Found actor-1: controllers
Found actor-2: Adaptation controller
Found actor-3: Evolution controller
Found actor-4: modules
Found actor-5: controller characteristics
Found actor-6: AN intent
Found actor-7: evolution algorithms
Found actor-8: underlay specific APIs
Found actor-9: edge controllers


### FG-AN-USECASE-009 cat2


In [67]:
app.create_actors_relationship_with_usecase("Emergency management controller", "uses", "controllers", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("controllers", "detect", "emergencies", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("controllers", "optimize", "network resource allocation", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("Emergency management controller", "instantiates", "controllers", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("Emergency management controller", "input", "intent", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("Emergency management controller", "output", "intent", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("controllers", "need", "network domain", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("intent", "has", "ml pipeline", "usecase_009_cat2")


Created node: Emergency management controller
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1']
adding label usecase_009_cat2 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1']
Created relation: Emergency management controller - uses - controllers in usecase_009_cat2
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2']
ignoring ...
Created node: emergencies
Created relation: controllers - detect - emergencies in usecase_009_cat2
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2']
ignoring ...
Created node: network

In [68]:
app.find_all_usecase_actors_label("usecase_009_cat2")

Found actor-1: controllers
Found actor-2: intent
Found actor-3: Emergency management controller
Found actor-4: emergencies
Found actor-5: network resource allocation
Found actor-6: network domain
Found actor-7: ml pipeline


### FG-AN-USECASE-010 cat1

In [69]:
app.create_actors_relationship_with_usecase("AN orchestrator", "accepts", "AN intent", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Evolution controller", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Adaptation controller", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Experimentation controller", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN intent", "has", "vertical specific inputs", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Experimentation controller", "validates", "controllers", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", "controllers", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "produces", "reports", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("underlay", "are", "CI/CD pipeline", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "reference points", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("underlay", "has", "reference points", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "controllers", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay capabilities", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("underlay", "has", "underlay capabilities", "usecase_010_cat1")



Created node: AN orchestrator
Node already exists: AN intent in ['usecase_009_cat1']
adding label usecase_010_cat1 in addition to ['usecase_009_cat1']
Created relation: AN orchestrator - accepts - AN intent in usecase_010_cat1
Node already exists: AN orchestrator in ['usecase_010_cat1']
ignoring ...
Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1']
adding label usecase_010_cat1 in addition to ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1']
Created relation: AN orchestrator - configures - Evolution controller in usecase_010_cat1
Node already exists: AN orchestrator in ['usecase_010_cat1']
ignoring ...
Node already exists: Adaptation controller in ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1']
adding label usecase_010_cat1 in addition to ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1']
Created relation: AN orchestrator - configures - Adaptation controller in usecase_010_cat1

In [70]:
app.find_all_usecase_actors_label("usecase_010_cat1")

Found actor-1: controllers
Found actor-2: reports
Found actor-3: Experimentation controller
Found actor-4: Adaptation controller
Found actor-5: underlay
Found actor-6: Evolution controller
Found actor-7: AN intent
Found actor-8: AN orchestrator
Found actor-9: vertical specific inputs
Found actor-10: CI/CD pipeline
Found actor-11: reference points
Found actor-12: underlay capabilities


### FG-AN-USECASE-010 cat2

In [71]:
app.create_actors_relationship_with_usecase("Cloud service controller", "uses", "controllers", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "intent", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "simulation", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "validation", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("validation", "needs", "KPI", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("validation", "needs", "AN sandbox", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "output", "cloud service deployment", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("cloud service deployment", "needs", "optimization", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "output", "network service deployment", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("network service deployment", "needs", "optimization", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "monitors", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("monitors", "output", "reports", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("network service deployment", "needs", "technology migration", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("cloud service deployment", "needs", "technology migration", "usecase_010_cat2")




Created node: Cloud service controller
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1']
adding label usecase_010_cat2 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1']
Created relation: Cloud service controller - uses - controllers in usecase_010_cat2
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2']
ignoring ...
Node already exists: intent in ['usecase_005', 'usecase_009_cat2']
adding label usecase_010_cat2 in addition to ['usecase_005', 'usecase_009_cat2']
Created relation: controllers - needs - intent in usecase_010_cat2

In [72]:
app.find_all_usecase_actors_label("usecase_010_cat2")

Found actor-1: controllers
Found actor-2: reports
Found actor-3: AN sandbox
Found actor-4: intent
Found actor-5: KPI
Found actor-6: optimization
Found actor-7: Cloud service controller
Found actor-8: simulation
Found actor-9: validation
Found actor-10: cloud service deployment
Found actor-11: network service deployment
Found actor-12: monitors
Found actor-13: technology migration


### FG-AN-USECASE-011 cat1

In [73]:
app.create_actors_relationship_with_usecase("AN orchestrator", "input", "intent", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "domain orchestrator", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Evolution controller", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Adaptation controller", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("intent", "has", "controller design", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("intent", "has", "controller deployment", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("intent", "has", "controller management", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("controllers", "monitors", "inter-domain-connectivity", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("controllers", "configures", "inter-domain-connectivity", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("controllers", "manages", "inter-domain-connectivity", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("Evolution controller", "implements", "evolution algorithms", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("evolution algorithms", "implements", "evolution", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("controllers", "need", "evolution", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("underlay", "are", "inter-domain-connectivity", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "controllers", "usecase_011_cat1")

Node already exists: AN orchestrator in ['usecase_010_cat1']
adding label usecase_011_cat1 in addition to ['usecase_010_cat1']
Node already exists: intent in ['usecase_005', 'usecase_009_cat2', 'usecase_010_cat2']
adding label usecase_011_cat1 in addition to ['usecase_005', 'usecase_009_cat2', 'usecase_010_cat2']
Created relation: AN orchestrator - input - intent in usecase_011_cat1
Node already exists: AN orchestrator in ['usecase_010_cat1', 'usecase_011_cat1']
ignoring ...
Created node: domain orchestrator
Created relation: AN orchestrator - configures - domain orchestrator in usecase_011_cat1
Node already exists: AN orchestrator in ['usecase_010_cat1', 'usecase_011_cat1']
ignoring ...
Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1']
adding label usecase_011_cat1 in addition to ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1']
Created relation: AN orchestrator - configures - Ev

In [74]:
app.find_all_usecase_actors_label("usecase_011_cat1")

Found actor-1: controllers
Found actor-2: Adaptation controller
Found actor-3: underlay
Found actor-4: intent
Found actor-5: Evolution controller
Found actor-6: evolution algorithms
Found actor-7: AN orchestrator
Found actor-8: domain orchestrator
Found actor-9: controller design
Found actor-10: controller deployment
Found actor-11: controller management
Found actor-12: inter-domain-connectivity
Found actor-13: evolution


### FG-AN-USECASE-011 cat2

In [75]:
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "has", "alternatives", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("verticals", "provide", "service", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("Rural sites", "need", "service", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("verticals", "need", "inter-domain-connectivity", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("Rural sites", "need", "inter-domain-connectivity", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "is", "shared", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "need", "edge compute", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "need", "CL automation framework", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("edge compute", "need", "CL automation framework", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("edge compute", "has", "micro service", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("alternatives", "has", "heterogeneous", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "controllers", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "need", "management", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "onboarding", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("applications", "need", "onboarding", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "monitoring", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("applications", "need", "monitoring", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "report", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("human operator", "need", "report", "usecase_011_cat2")

Node already exists: inter-domain-connectivity in ['usecase_011_cat1']
adding label usecase_011_cat2 in addition to ['usecase_011_cat1']
Created node: alternatives
Created relation: inter-domain-connectivity - has - alternatives in usecase_011_cat2
Created node: verticals
Created node: service
Created relation: verticals - provide - service in usecase_011_cat2
Created node: Rural sites
Node already exists: service in ['usecase_011_cat2']
ignoring ...
Created relation: Rural sites - need - service in usecase_011_cat2
Node already exists: verticals in ['usecase_011_cat2']
ignoring ...
Node already exists: inter-domain-connectivity in ['usecase_011_cat1', 'usecase_011_cat2']
ignoring ...
Created relation: verticals - need - inter-domain-connectivity in usecase_011_cat2
Node already exists: Rural sites in ['usecase_011_cat2']
ignoring ...
Node already exists: inter-domain-connectivity in ['usecase_011_cat1', 'usecase_011_cat2']
ignoring ...
Created relation: Rural sites - need - inter-doma

In [76]:
app.find_all_usecase_actors_label("usecase_011_cat2")

Found actor-1: controllers
Found actor-2: CL automation framework
Found actor-3: underlay orchestrator
Found actor-4: inter-domain-connectivity
Found actor-5: alternatives
Found actor-6: verticals
Found actor-7: service
Found actor-8: Rural sites
Found actor-9: shared
Found actor-10: edge compute
Found actor-11: micro service
Found actor-12: heterogeneous
Found actor-13: management
Found actor-14: onboarding
Found actor-15: applications
Found actor-16: monitoring
Found actor-17: report
Found actor-18: human operator


### FG-AN-USECASE-012 cat1

In [77]:
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", "underlay capabilities", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay capabilities", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("underlay capabilities", "is", "underlay configuration", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("underlay capabilities", "is", "monitoring", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "can", "recommendation", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("recommendation", "is", "underlay capabilities", "usecase_012_cat1")

Node already exists: Adaptation controller in ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1']
adding label usecase_012_cat1 in addition to ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1']
Node already exists: underlay capabilities in ['usecase_010_cat1']
adding label usecase_012_cat1 in addition to ['usecase_010_cat1']
Created relation: Adaptation controller - monitors - underlay capabilities in usecase_012_cat1
Node already exists: Adaptation controller in ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1']
ignoring ...
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1']
ignoring ...
Created relation: Adaptation controller - discovers - underlay capabilities in usecase_012_cat1
Node already exists: underlay capabilities in ['usecase_010_cat1', 'useca

In [78]:
app.find_all_usecase_actors_label("usecase_012_cat1")

Found actor-1: Adaptation controller
Found actor-2: Evolution controller
Found actor-3: underlay capabilities
Found actor-4: monitoring
Found actor-5: underlay configuration
Found actor-6: recommendation


### FG-AN-USECASE-012 cat2

In [79]:
app.create_actors_relationship_with_usecase("Data analysis function", "has", "data collector", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("Network function", "provide", "event report", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "UE IP address change", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "UP path change", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "PDU Session Establishment", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "PDU Session Release", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("Data analysis function", "", "Data analytics", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("Data analytics", "is", "Load information of UPF", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "data", "usecase_012_cat2")

Created node: Data analysis function
Created node: data collector
Created relation: Data analysis function - has - data collector in usecase_012_cat2
Created node: Network function
Created node: event report
Created relation: Network function - provide - event report in usecase_012_cat2
Node already exists: event report in ['usecase_012_cat2']
ignoring ...
Created node: UE IP address change
Created relation: event report - is - UE IP address change in usecase_012_cat2
Node already exists: event report in ['usecase_012_cat2']
ignoring ...
Created node: UP path change
Created relation: event report - is - UP path change in usecase_012_cat2
Node already exists: event report in ['usecase_012_cat2']
ignoring ...
Created node: PDU Session Establishment
Created relation: event report - is - PDU Session Establishment in usecase_012_cat2
Node already exists: event report in ['usecase_012_cat2']
ignoring ...
Created node: PDU Session Release
Created relation: event report - is - PDU Session Rele

In [80]:
app.find_all_usecase_actors_label("usecase_012_cat2")

Found actor-1: data
Found actor-2: Data analytics
Found actor-3: Data analysis function
Found actor-4: data collector
Found actor-5: Network function
Found actor-6: event report
Found actor-7: UE IP address change
Found actor-8: UP path change
Found actor-9: PDU Session Establishment
Found actor-10: PDU Session Release
Found actor-11: Load information of UPF


### FG-AN-USECASE-013 cat1

In [81]:

app.create_actors_relationship_with_usecase("Network function", "has", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay requirements", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "provide", "recommendation", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("recommendation", "is", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("recommendation", "is", "underlay requirements", "usecase_013_cat1")

Node already exists: Network function in ['usecase_012_cat2']
adding label usecase_013_cat1 in addition to ['usecase_012_cat2']
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1']
adding label usecase_013_cat1 in addition to ['usecase_010_cat1', 'usecase_012_cat1']
Created relation: Network function - has - underlay capabilities in usecase_013_cat1
Node already exists: Adaptation controller in ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1']
adding label usecase_013_cat1 in addition to ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1']
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1']
ignoring ...
Created relation: Adaptation controller - monitors - underlay capabilities in usecase_013_cat1
Node already exists: Adaptation controller in 

In [82]:
app.find_all_usecase_actors_label("usecase_013_cat1")

Found actor-1: Adaptation controller
Found actor-2: Evolution controller
Found actor-3: underlay capabilities
Found actor-4: recommendation
Found actor-5: Network function
Found actor-6: underlay requirements


### FG-AN-USECASE-013 cat2

In [83]:
app.create_actors_relationship_with_usecase("controllers", "need", "provisioning", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("provisioning", "is", "multi-domain", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("provisioning", "need", "underlay capabilities", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "need", "plug and play", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "is", "Network function", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "is", "network slice", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "is", "network service", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "provide", "data collection", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("controllers", "need", "bootstrap", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network infrastructure", "need", "bootstrap", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network service", "need", "bootstrap", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("applications", "need", "bootstrap", "usecase_013_cat2")

Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2']
adding label usecase_013_cat2 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2']
Created node: provisioning
Created relation: controllers - need - provisioning in usecase_013_cat2
Node already exists: provisioning in ['usecase_013_cat2']
ignoring ...
Created node: multi-domain
Created relation: provisioning - is - multi-domain in usecase_013_cat2
Node already exists: provisioning in ['usecase_013_cat2']
ignoring ...
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1']
adding labe

In [84]:
app.find_all_usecase_actors_label("usecase_013_cat2")

Found actor-1: controllers
Found actor-2: underlay capabilities
Found actor-3: applications
Found actor-4: Network function
Found actor-5: provisioning
Found actor-6: multi-domain
Found actor-7: network instance
Found actor-8: plug and play
Found actor-9: network slice
Found actor-10: network service
Found actor-11: data collection
Found actor-12: bootstrap
Found actor-13: network infrastructure


### FG-AN-USECASE-014

In [85]:
app.create_actors_relationship_with_usecase("underlay", "are", "hybrid", "usecase_014")
app.create_actors_relationship_with_usecase("hybrid", "is", "simulators", "usecase_014")
app.create_actors_relationship_with_usecase("hybrid", "is", "test network", "usecase_014")
app.create_actors_relationship_with_usecase("hybrid", "is", "real network", "usecase_014")
app.create_actors_relationship_with_usecase("multi-domain", "is", "hybrid", "usecase_014")

Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1']
adding label usecase_014 in addition to ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1']
Created node: hybrid
Created relation: underlay - are - hybrid in usecase_014
Node already exists: hybrid in ['usecase_014']
ignoring ...
Node already exists: simulators in ['usecase_002']
adding label usecase_014 in addition to ['usecase_002']
Created relation: hybrid - is - simulators in usecase_014
Node already exists: hybrid in ['usecase_014']
ignoring ...
Created node: test network
Created relation: hybrid - is - test network in usecase_014
Node already exists: hybrid in ['usecase_014']
ignoring ...
Created node: real network
Created relation: hybrid - is - real network in usecase_014
Node already exists: multi-domain in ['usecase_013_cat2']
adding label usecase_014 in addition to ['usecase_013_cat2']
Node already exists: hybrid in ['usecase_014']
ignoring ...
Created relation: multi-domain - is - h

In [86]:
app.find_all_usecase_actors_label("usecase_014")

Found actor-1: simulators
Found actor-2: underlay
Found actor-3: multi-domain
Found actor-4: hybrid
Found actor-5: test network
Found actor-6: real network


### FG-AN-USECASE-015

*detailed implementation of this UC may be provided by Team "Avengers"*

In [87]:
app.create_actors_relationship_with_usecase("data", "is", "logs", "usecase_015")
app.create_actors_relationship_with_usecase("network instance", "has", "logs", "usecase_015")
app.create_actors_relationship_with_usecase("model repository", "has", "ML model", "usecase_015")
app.create_actors_relationship_with_usecase("model repository", "is", "3rd party", "usecase_015")


Node already exists: data in ['usecase_006_cat2', 'usecase_008_cat1', 'usecase_012_cat2']
adding label usecase_015 in addition to ['usecase_006_cat2', 'usecase_008_cat1', 'usecase_012_cat2']
Created node: logs
Created relation: data - is - logs in usecase_015
Node already exists: network instance in ['usecase_013_cat2']
adding label usecase_015 in addition to ['usecase_013_cat2']
Node already exists: logs in ['usecase_015']
ignoring ...
Created relation: network instance - has - logs in usecase_015
Created node: model repository
Node already exists: ML model in ['usecase_001', 'usecase_006_cat2', 'usecase_007_cat1']
adding label usecase_015 in addition to ['usecase_001', 'usecase_006_cat2', 'usecase_007_cat1']
Created relation: model repository - has - ML model in usecase_015
Node already exists: model repository in ['usecase_015']
ignoring ...
Created node: 3rd party
Created relation: model repository - is - 3rd party in usecase_015


In [88]:
app.find_all_usecase_actors_label("usecase_015")

Found actor-1: ML model
Found actor-2: data
Found actor-3: network instance
Found actor-4: logs
Found actor-5: model repository
Found actor-6: 3rd party


### FG-AN-USECASE-016

In [89]:
app.create_actors_relationship_with_usecase("underlay orchestrator", "process", "underlay intent", "usecase_016")
app.create_actors_relationship_with_usecase("underlay intent", "has", "network instance", "usecase_016")
app.create_actors_relationship_with_usecase("intent", "is", "underlay intent", "usecase_016")
app.create_actors_relationship_with_usecase("intent", "is", "AN intent", "usecase_016")
app.create_actors_relationship_with_usecase("underlay orchestrator", "interfaces", "AN orchestrator", "usecase_016")
app.create_actors_relationship_with_usecase("controllers", "is", "multi-domain", "usecase_016")
app.create_actors_relationship_with_usecase("underlay orchestrator", "manages", "network instance", "usecase_016")
app.create_actors_relationship_with_usecase("orchestrator", "export", "intent", "usecase_016")
app.create_actors_relationship_with_usecase("orchestrator", "import", "intent", "usecase_016")

app.create_actors_relationship_with_usecase("orchestrator", "is", "AN orchestrator", "usecase_016")
app.create_actors_relationship_with_usecase("orchestrator", "is", "underlay orchestrator", "usecase_016")
app.create_actors_relationship_with_usecase("AN intent", "has", "controller  definitions", "usecase_016")
app.create_actors_relationship_with_usecase("underlay intent", "has", "network instance definitions", "usecase_016")

Node already exists: underlay orchestrator in ['usecase_007_cat1', 'usecase_011_cat2']
adding label usecase_016 in addition to ['usecase_007_cat1', 'usecase_011_cat2']
Created node: underlay intent
Created relation: underlay orchestrator - process - underlay intent in usecase_016
Node already exists: underlay intent in ['usecase_016']
ignoring ...
Node already exists: network instance in ['usecase_013_cat2', 'usecase_015']
adding label usecase_016 in addition to ['usecase_013_cat2', 'usecase_015']
Created relation: underlay intent - has - network instance in usecase_016
Node already exists: intent in ['usecase_005', 'usecase_009_cat2', 'usecase_010_cat2', 'usecase_011_cat1']
adding label usecase_016 in addition to ['usecase_005', 'usecase_009_cat2', 'usecase_010_cat2', 'usecase_011_cat1']
Node already exists: underlay intent in ['usecase_016']
ignoring ...
Created relation: intent - is - underlay intent in usecase_016
Node already exists: intent in ['usecase_005', 'usecase_009_cat2', '

In [90]:
app.find_all_usecase_actors_label("usecase_016")

Found actor-1: controllers
Found actor-2: intent
Found actor-3: underlay orchestrator
Found actor-4: AN intent
Found actor-5: AN orchestrator
Found actor-6: multi-domain
Found actor-7: network instance
Found actor-8: underlay intent
Found actor-9: orchestrator
Found actor-10: controller  definitions
Found actor-11: network instance definitions


### FG-AN-USECASE-017

In [91]:
app.create_actors_relationship_with_usecase("data", "is", "user experience", "usecase_017")
app.create_actors_relationship_with_usecase("data collection", "has", "subjective tests", "usecase_017")
app.create_actors_relationship_with_usecase("user experience", "need", "subjective tests", "usecase_017")
app.create_actors_relationship_with_usecase("subjective tests", "has", "user selection", "usecase_017")
app.create_actors_relationship_with_usecase("subjective tests", "has", "location selection", "usecase_017")
app.create_actors_relationship_with_usecase("applications", "need", "subjective tests", "usecase_017")
app.create_actors_relationship_with_usecase("applications", "has", "KPI", "usecase_017")
app.create_actors_relationship_with_usecase("underlay", "has", "measurements", "usecase_017")
app.create_actors_relationship_with_usecase("subjective tests", "provide","user opinion", "usecase_017")


app.create_actors_relationship_with_usecase("QoE prediction controller", "is", "controllers", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction controller", "need", "KPI", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction controller", "provide", "optimization", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction controller", "need", "user behavior", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction controller", "need", "KPI", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction controller", "need", "ML model", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction controller", "need", "subjective tests", "usecase_017")

app.create_actors_relationship_with_usecase("ML model", "maps", "QoE metrics", "usecase_017")
app.create_actors_relationship_with_usecase("ML model", "maps", "measurements", "usecase_017")
app.create_actors_relationship_with_usecase("underlay orchestrator", "discover", "measurements", "usecase_017")


Node already exists: data in ['usecase_006_cat2', 'usecase_008_cat1', 'usecase_012_cat2', 'usecase_015']
adding label usecase_017 in addition to ['usecase_006_cat2', 'usecase_008_cat1', 'usecase_012_cat2', 'usecase_015']
Node already exists: user experience in ['usecase_008_cat2']
adding label usecase_017 in addition to ['usecase_008_cat2']
Created relation: data - is - user experience in usecase_017
Node already exists: data collection in ['usecase_013_cat2']
adding label usecase_017 in addition to ['usecase_013_cat2']
Created node: subjective tests
Created relation: data collection - has - subjective tests in usecase_017
Node already exists: user experience in ['usecase_008_cat2', 'usecase_017']
ignoring ...
Node already exists: subjective tests in ['usecase_017']
ignoring ...
Created relation: user experience - need - subjective tests in usecase_017
Node already exists: subjective tests in ['usecase_017']
ignoring ...
Created node: user selection
Created relation: subjective tests -

In [92]:
app.find_all_usecase_actors_label("usecase_017")

Found actor-1: controllers
Found actor-2: ML model
Found actor-3: underlay
Found actor-4: data
Found actor-5: KPI
Found actor-6: optimization
Found actor-7: underlay orchestrator
Found actor-8: user experience
Found actor-9: applications
Found actor-10: data collection
Found actor-11: subjective tests
Found actor-12: user selection
Found actor-13: location selection
Found actor-14: measurements
Found actor-15: user opinion
Found actor-16: QoE prediction controller
Found actor-17: user behavior
Found actor-18: QoE metrics


### FG-AN-USECASE-018

In [93]:
app.create_actors_relationship_with_usecase("AN intent", "has", "outer loop", "usecase_018")
app.create_actors_relationship_with_usecase("AN intent", "has", "deployment preferences", "usecase_018")
app.create_actors_relationship_with_usecase("AN orchestrator", "requires", "AN intent", "usecase_018")
app.create_actors_relationship_with_usecase("AN orchestrator", "derives", "sub-intent", "usecase_018")
app.create_actors_relationship_with_usecase("sub-intent", "is", "intent", "usecase_018")
app.create_actors_relationship_with_usecase("sub-intent", "has", "inner loop", "usecase_018")
app.create_actors_relationship_with_usecase("inner loop", "is", "controller", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "is", "controller", "usecase_018")
app.create_actors_relationship_with_usecase("AN orchestrator", "optimizes", "intent", "usecase_018")
app.create_actors_relationship_with_usecase("AN orchestrator", "requires", "feedback", "usecase_018")
app.create_actors_relationship_with_usecase("Adaptation controller", "provides", "feedback", "usecase_018")

app.create_actors_relationship_with_usecase("outer loop", "has", "QoS QoE metrics", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "deployment preferences", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "ML model", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "cache policy", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "scaling policy", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "edge selection", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "storage configuration", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "API selection", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "concurrency selection", "usecase_018")
app.create_actors_relationship_with_usecase("experiments", "is", "data measurement", "usecase_018")
app.create_actors_relationship_with_usecase("experiments", "is", "policy selection", "usecase_018")
app.create_actors_relationship_with_usecase("inner loop", "has", "CDN configuration", "usecase_018")
app.create_actors_relationship_with_usecase("inner loop", "has", "QoE control", "usecase_018")
app.create_actors_relationship_with_usecase("Evolution controller", "evolve", "intent", "usecase_018")

Node already exists: AN intent in ['usecase_009_cat1', 'usecase_010_cat1', 'usecase_016']
adding label usecase_018 in addition to ['usecase_009_cat1', 'usecase_010_cat1', 'usecase_016']
Created node: outer loop
Created relation: AN intent - has - outer loop in usecase_018
Node already exists: AN intent in ['usecase_009_cat1', 'usecase_010_cat1', 'usecase_016', 'usecase_018']
ignoring ...
Created node: deployment preferences
Created relation: AN intent - has - deployment preferences in usecase_018
Node already exists: AN orchestrator in ['usecase_010_cat1', 'usecase_011_cat1', 'usecase_016']
adding label usecase_018 in addition to ['usecase_010_cat1', 'usecase_011_cat1', 'usecase_016']
Node already exists: AN intent in ['usecase_009_cat1', 'usecase_010_cat1', 'usecase_016', 'usecase_018']
ignoring ...
Created relation: AN orchestrator - requires - AN intent in usecase_018
Node already exists: AN orchestrator in ['usecase_010_cat1', 'usecase_011_cat1', 'usecase_016', 'usecase_018']
ignor

In [94]:
app.find_all_usecase_actors_label("usecase_018")

Found actor-1: ML model
Found actor-2: experiments
Found actor-3: feedback
Found actor-4: Adaptation controller
Found actor-5: intent
Found actor-6: Evolution controller
Found actor-7: AN intent
Found actor-8: AN orchestrator
Found actor-9: outer loop
Found actor-10: deployment preferences
Found actor-11: sub-intent
Found actor-12: inner loop
Found actor-13: controller
Found actor-14: QoS QoE metrics
Found actor-15: cache policy
Found actor-16: scaling policy
Found actor-17: edge selection
Found actor-18: storage configuration
Found actor-19: API selection
Found actor-20: concurrency selection
Found actor-21: data measurement
Found actor-22: policy selection
Found actor-23: CDN configuration
Found actor-24: QoE control


### FG-AN-USECASE-019

In [95]:
app.create_actors_relationship_with_usecase("underlay", "has", "underlay specific APIs", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "underlay specific configurations", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "multi-domain", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "network instance", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "network instance", "usecase_019")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay specific configurations", "usecase_019")

app.create_actors_relationship_with_usecase("underlay", "has", "architecture options", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "deployment options", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "data model", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "applications", "usecase_019")
app.create_actors_relationship_with_usecase("applications", "need", "multi-domain", "usecase_019")
app.create_actors_relationship_with_usecase("applications", "need", "real time", "usecase_019")
app.create_actors_relationship_with_usecase("applications", "need", "non real time", "usecase_019")
app.create_actors_relationship_with_usecase("underlay orchestrator", "provide", "controller placement", "usecase_019")
app.create_actors_relationship_with_usecase("controller placement", "is", "multi-domain", "usecase_019")

Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017']
adding label usecase_019 in addition to ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017']
Node already exists: underlay specific APIs in ['usecase_009_cat1']
adding label usecase_019 in addition to ['usecase_009_cat1']
Created relation: underlay - has - underlay specific APIs in usecase_019
Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019']
ignoring ...
Created node: underlay specific configurations
Created relation: underlay - has - underlay specific configurations in usecase_019
Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019']
ignoring ...
Node already exists: multi-domain in ['usecase_013_cat2', 'usecase_014', 'usecase_016']
adding label usecase_019 in addition to [

In [96]:
app.find_all_usecase_actors_label("usecase_019")

Found actor-1: Adaptation controller
Found actor-2: underlay
Found actor-3: real time
Found actor-4: underlay orchestrator
Found actor-5: underlay specific APIs
Found actor-6: applications
Found actor-7: multi-domain
Found actor-8: network instance
Found actor-9: underlay specific configurations
Found actor-10: architecture options
Found actor-11: deployment options
Found actor-12: data model
Found actor-13: non real time
Found actor-14: controller placement


### FG-AN-USECASE-020

In [97]:
app.create_actors_relationship_with_usecase("edge network", "is", "underlay", "usecase_020")
app.create_actors_relationship_with_usecase("edge network", "has", "Data analytics", "usecase_020")
app.create_actors_relationship_with_usecase("Data analytics", "is", "real time", "usecase_020")
app.create_actors_relationship_with_usecase("Data analytics", "is", "non real time", "usecase_020")
app.create_actors_relationship_with_usecase("verticals", "require", "Data analytics", "usecase_020")
app.create_actors_relationship_with_usecase("edge network", "has", "applications", "usecase_020")
app.create_actors_relationship_with_usecase("verticals", "require", "applications", "usecase_020")
app.create_actors_relationship_with_usecase("applications", "require", "optimization", "usecase_020")

Created node: edge network
Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019']
adding label usecase_020 in addition to ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019']
Created relation: edge network - is - underlay in usecase_020
Node already exists: edge network in ['usecase_020']
ignoring ...
Node already exists: Data analytics in ['usecase_006_cat2', 'usecase_012_cat2']
adding label usecase_020 in addition to ['usecase_006_cat2', 'usecase_012_cat2']
Created relation: edge network - has - Data analytics in usecase_020
Node already exists: Data analytics in ['usecase_006_cat2', 'usecase_012_cat2', 'usecase_020']
ignoring ...
Node already exists: real time in ['usecase_006_cat2', 'usecase_019']
adding label usecase_020 in addition to ['usecase_006_cat2', 'usecase_019']
Created relation: Data analytics - is - real time in usecase_020
Node already exists: 

In [98]:
app.find_all_usecase_actors_label("usecase_020")

Found actor-1: underlay
Found actor-2: real time
Found actor-3: Data analytics
Found actor-4: optimization
Found actor-5: verticals
Found actor-6: applications
Found actor-7: non real time
Found actor-8: edge network


### FG-AN-USECASE-021

In [99]:
app.create_actors_relationship_with_usecase("Experimentation controller", "creates", "experimentation strategies", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "executes", "experimentation strategies", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "optimizes", "experimentation strategies", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "optimizes", "experimentation strategies", "usecase_021")
app.create_actors_relationship_with_usecase("AN sandbox", "is", "underlay", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "monitors", "experiments", "usecase_021")

app.create_actors_relationship_with_usecase("Experimentation controller", "need", "data", "usecase_021")
app.create_actors_relationship_with_usecase("rare scenarios", "is", "experiments", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "creates", "optimization", "usecase_021")

Node already exists: Experimentation controller in ['usecase_002', 'usecase_010_cat1']
adding label usecase_021 in addition to ['usecase_002', 'usecase_010_cat1']
Created node: experimentation strategies
Created relation: Experimentation controller - creates - experimentation strategies in usecase_021
Node already exists: Experimentation controller in ['usecase_002', 'usecase_010_cat1', 'usecase_021']
ignoring ...
Node already exists: experimentation strategies in ['usecase_021']
ignoring ...
Created relation: Experimentation controller - executes - experimentation strategies in usecase_021
Node already exists: Experimentation controller in ['usecase_002', 'usecase_010_cat1', 'usecase_021']
ignoring ...
Node already exists: experimentation strategies in ['usecase_021']
ignoring ...
Created relation: Experimentation controller - optimizes - experimentation strategies in usecase_021
Node already exists: Experimentation controller in ['usecase_002', 'usecase_010_cat1', 'usecase_021']
igno

In [100]:
app.find_all_usecase_actors_label("usecase_021")

Found actor-1: Experimentation controller
Found actor-2: AN sandbox
Found actor-3: experiments
Found actor-4: underlay
Found actor-5: data
Found actor-6: optimization
Found actor-7: experimentation strategies
Found actor-8: rare scenarios


### FG-AN-USECASE-022

In [101]:
app.create_actors_relationship_with_usecase("underlay capabilities", "has", "domain specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("underlay capabilities", "has", "network service specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("domain specific aspects", "has", "multi-domain", "usecase_022")
app.create_actors_relationship_with_usecase("domain specific aspects", "has", "unknown", "usecase_022")
app.create_actors_relationship_with_usecase("underlay capabilities", "has", "service specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("service specific aspects", "has", "E2E", "usecase_022")
app.create_actors_relationship_with_usecase("service specific aspects", "has", "domain specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("service description", "has", "service specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("service specific aspects", "has", "sub-problems", "usecase_022")
app.create_actors_relationship_with_usecase("underlay orchestrator", "discovers", "service specific aspects", "usecase_022")

app.create_actors_relationship_with_usecase("service evolution", "solve", "sub-problems", "usecase_022")
app.create_actors_relationship_with_usecase("Evolution controller", "has", "service evolution", "usecase_022")
app.create_actors_relationship_with_usecase("service evolution", "has", "service design", "usecase_022")
app.create_actors_relationship_with_usecase("service evolution", "has", "service optimization", "usecase_022")


Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_013_cat2']
adding label usecase_022 in addition to ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_013_cat2']
Created node: domain specific aspects
Created relation: underlay capabilities - has - domain specific aspects in usecase_022
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_013_cat2', 'usecase_022']
ignoring ...
Created node: network service specific aspects
Created relation: underlay capabilities - has - network service specific aspects in usecase_022
Node already exists: domain specific aspects in ['usecase_022']
ignoring ...
Node already exists: multi-domain in ['usecase_013_cat2', 'usecase_014', 'usecase_016', 'usecase_019']
adding label usecase_022 in addition to ['usecase_013_cat2', 'usecase_014', 'usecase_016', 'usecase_019']
Created relation: domain specific aspects -

In [102]:
app.find_all_usecase_actors_label("usecase_022")

Found actor-1: Evolution controller
Found actor-2: underlay orchestrator
Found actor-3: underlay capabilities
Found actor-4: multi-domain
Found actor-5: domain specific aspects
Found actor-6: network service specific aspects
Found actor-7: unknown
Found actor-8: service specific aspects
Found actor-9: E2E
Found actor-10: service description
Found actor-11: sub-problems
Found actor-12: service evolution
Found actor-13: service design
Found actor-14: service optimization


### FG-AN-USECASE-023
*detailed implementation of this UC may be provided by Team Costa Rica*

In [103]:
app.create_actors_relationship_with_usecase("controllers", "has", "ML model", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "has", "SINK", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "is", "autonomous", "usecase_023")
app.create_actors_relationship_with_usecase("data", "is", "environment data", "usecase_023")
app.create_actors_relationship_with_usecase("SINK", "has", "underlay specific configurations", "usecase_023")
app.create_actors_relationship_with_usecase("underlay specific configurations", "is", "tx power", "usecase_023")
app.create_actors_relationship_with_usecase("data", "is", "multi mode", "usecase_023")
app.create_actors_relationship_with_usecase("data", "is", "multi-mode", "usecase_023")
app.create_actors_relationship_with_usecase("multi-mode", "is", "image", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "has", "SRC", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "discover", "SRC", "usecase_023")
app.create_actors_relationship_with_usecase("root cause analysis", "is", "ML model", "usecase_023")
app.create_actors_relationship_with_usecase("root cause analysis", "has", "failure", "usecase_023")
app.create_actors_relationship_with_usecase("failure", "has", "software failure", "usecase_023")
app.create_actors_relationship_with_usecase("failure", "has", "hardware failure", "usecase_023")
app.create_actors_relationship_with_usecase("failure", "has", "middleware failure", "usecase_023")
app.create_actors_relationship_with_usecase("underlay", "has", "version", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "has", "version", "usecase_023")

app.create_actors_relationship_with_usecase("sub-problems", "has", "runtime", "usecase_023")


Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017']
adding label usecase_023 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017']
Node already exists: ML model in ['usecase_001', 'usecase_006_cat2', 'usecase_007_cat1', 'usecase_015', 'usecase_017', 'usecase_018']
adding label usecase_023 in addition to ['usecase_001', 'usecase_006_cat2', 'usecase_007_cat1', 'usecase_015', 'usecase_017', 'usecase_018']
Created relation: controllers - has - ML model in usecase_023
Node already exists: cont

In [104]:
app.find_all_usecase_actors_label("usecase_023")

Found actor-1: controllers
Found actor-2: ML model
Found actor-3: underlay
Found actor-4: data
Found actor-5: underlay specific configurations
Found actor-6: sub-problems
Found actor-7: SINK
Found actor-8: autonomous
Found actor-9: environment data
Found actor-10: tx power
Found actor-11: multi mode
Found actor-12: multi-mode
Found actor-13: image
Found actor-14: SRC
Found actor-15: root cause analysis
Found actor-16: failure
Found actor-17: software failure
Found actor-18: hardware failure
Found actor-19: middleware failure
Found actor-20: version
Found actor-21: runtime


### FG-AN-USECASE-024

In [105]:
app.create_actors_relationship_with_usecase("underlay", "is", "adaptive hardware", "usecase_024")
app.create_actors_relationship_with_usecase("design", "has", "tradeoff", "usecase_024")
app.create_actors_relationship_with_usecase("controllers", "has", "design", "usecase_024")
app.create_actors_relationship_with_usecase("Evolution controller", "optimizes", "design", "usecase_024")

app.create_actors_relationship_with_usecase("edge network", "has", "ML model", "usecase_024")
app.create_actors_relationship_with_usecase("edge network", "need", "low latency", "usecase_024")
app.create_actors_relationship_with_usecase("edge network", "need", "low power consumption", "usecase_024")
app.create_actors_relationship_with_usecase("edge network", "need", "low memory", "usecase_024")

app.create_actors_relationship_with_usecase("FPGA", "is", "adaptive hardware", "usecase_024")
app.create_actors_relationship_with_usecase("adaptive System-on-Chip", "is", "adaptive hardware", "usecase_024")
app.create_actors_relationship_with_usecase("adaptive hardware", "provide", "customization", "usecase_024")
app.create_actors_relationship_with_usecase("tradeoff", "is", "concurrency", "usecase_024")
app.create_actors_relationship_with_usecase("tradeoff", "is", "buffering between layers", "usecase_024")
app.create_actors_relationship_with_usecase("tradeoff", "is", "offloading of layers", "usecase_024")
app.create_actors_relationship_with_usecase("tradeoff", "is", "precision", "usecase_024")
app.create_actors_relationship_with_usecase("tradeoff", "is", "performance", "usecase_024")
app.create_actors_relationship_with_usecase("tradeoff", "is", "energy efficiency", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "has", "model architecture", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "has", "intermediate representation ", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "need", "optimization", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "need", "hardware adaptation", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "need", "training", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "need", "evaluation", "usecase_024")

Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019', 'usecase_020', 'usecase_021', 'usecase_023']
adding label usecase_024 in addition to ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019', 'usecase_020', 'usecase_021', 'usecase_023']
Created node: adaptive hardware
Created relation: underlay - is - adaptive hardware in usecase_024
Created node: design
Created node: tradeoff
Created relation: design - has - tradeoff in usecase_024
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017', 'usecase_023']
adding label usecase_024 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007

In [106]:
app.find_all_usecase_actors_label("usecase_024")

Found actor-1: controllers
Found actor-2: ML model
Found actor-3: underlay
Found actor-4: Evolution controller
Found actor-5: optimization
Found actor-6: evaluation
Found actor-7: edge network
Found actor-8: adaptive hardware
Found actor-9: design
Found actor-10: tradeoff
Found actor-11: low latency
Found actor-12: low power consumption
Found actor-13: low memory
Found actor-14: FPGA
Found actor-15: adaptive System-on-Chip
Found actor-16: customization
Found actor-17: concurrency
Found actor-18: buffering between layers
Found actor-19: offloading of layers
Found actor-20: precision
Found actor-21: performance
Found actor-22: energy efficiency
Found actor-23: model architecture
Found actor-24: intermediate representation 
Found actor-25: hardware adaptation
Found actor-26: training


### FG-AN-USECASE-025

In [107]:
app.create_actors_relationship_with_usecase("evolution algorithms", "need", "ML model", "usecase_025")
app.create_actors_relationship_with_usecase("ML models", "need", "transfer learning", "usecase_025")
app.create_actors_relationship_with_usecase("Knowledge Base", "provide", "transfer learning", "usecase_025")
app.create_actors_relationship_with_usecase("AN orchestrator", "manages", "Knowledge Base", "usecase_025")

app.create_actors_relationship_with_usecase("environment data", "is", "data", "usecase_025")
app.create_actors_relationship_with_usecase("environment data", "is", "signal strength", "usecase_025")
app.create_actors_relationship_with_usecase("environment data", "is", "mobility data", "usecase_025")
app.create_actors_relationship_with_usecase("user model", "is", "data", "usecase_025")
app.create_actors_relationship_with_usecase("Adaptation controller", "need", "adaptation algorithms", "usecase_025")
app.create_actors_relationship_with_usecase("adaptation algorithms", "need", "validation", "usecase_025")
app.create_actors_relationship_with_usecase("controllers", "need", "adaptation algorithms", "usecase_025")
app.create_actors_relationship_with_usecase("underlay specific configurations", "is", "coverage parameters", "usecase_025")
app.create_actors_relationship_with_usecase("underlay specific configurations", "is", "beam parameters", "usecase_025")
app.create_actors_relationship_with_usecase("ML model", "need", "retraining", "usecase_025")
app.create_actors_relationship_with_usecase("simulators", "need", "controllers", "usecase_025")

Node already exists: evolution algorithms in ['usecase_009_cat1', 'usecase_011_cat1']
adding label usecase_025 in addition to ['usecase_009_cat1', 'usecase_011_cat1']
Node already exists: ML model in ['usecase_001', 'usecase_006_cat2', 'usecase_007_cat1', 'usecase_015', 'usecase_017', 'usecase_018', 'usecase_023', 'usecase_024']
adding label usecase_025 in addition to ['usecase_001', 'usecase_006_cat2', 'usecase_007_cat1', 'usecase_015', 'usecase_017', 'usecase_018', 'usecase_023', 'usecase_024']
Created relation: evolution algorithms - need - ML model in usecase_025
Created node: ML models
Created node: transfer learning
Created relation: ML models - need - transfer learning in usecase_025
Node already exists: Knowledge Base in ['usecase_001', 'usecase_002', 'usecase_003', 'usecase_004', 'usecase_006_cat1']
adding label usecase_025 in addition to ['usecase_001', 'usecase_002', 'usecase_003', 'usecase_004', 'usecase_006_cat1']
Node already exists: transfer learning in ['usecase_025']
i

In [108]:
app.find_all_usecase_actors_label("usecase_025")

Found actor-1: controllers
Found actor-2: ML model
Found actor-3: simulators
Found actor-4: Adaptation controller
Found actor-5: data
Found actor-6: evolution algorithms
Found actor-7: AN orchestrator
Found actor-8: validation
Found actor-9: underlay specific configurations
Found actor-10: environment data
Found actor-11: ML models
Found actor-12: transfer learning
Found actor-13: signal strength
Found actor-14: mobility data
Found actor-15: user model
Found actor-16: adaptation algorithms
Found actor-17: coverage parameters
Found actor-18: beam parameters
Found actor-19: retraining
Found actor-20: Knowledge Base


### FG-AN-USECASE-026

In [109]:
app.create_actors_relationship_with_usecase("Evolution controller", "manage", "evolvable controllers", "usecase_026")
app.create_actors_relationship_with_usecase("evolvable controllers", "is", "network instance", "usecase_026")
app.create_actors_relationship_with_usecase("evolvable controllers", "is", "controllers", "usecase_026")
app.create_actors_relationship_with_usecase("Evolution controller", "uses", "CI/CD pipeline", "usecase_026")
app.create_actors_relationship_with_usecase("Evolution controller", "uses", "CL automation framework", "usecase_026")

app.create_actors_relationship_with_usecase("underlay", "uses", "Evolution as a service", "usecase_026")
app.create_actors_relationship_with_usecase("Evolution as a service", "uses", "Evolution controller", "usecase_026")

Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024']
adding label usecase_026 in addition to ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024']
Created node: evolvable controllers
Created relation: Evolution controller - manage - evolvable controllers in usecase_026
Node already exists: evolvable controllers in ['usecase_026']
ignoring ...
Node already exists: network instance in ['usecase_013_cat2', 'usecase_015', 'usecase_016', 'usecase_019']
adding label usecase_026 in addition to ['usecase_013_cat2', 'usecase_015', 'usecase_016', 'usecase_019']
Created relation: evolvable controllers - is - network instance in usecase_026
Node already exists: evolvable controllers in ['useca

In [110]:
app.find_all_usecase_actors_label("usecase_026")

Found actor-1: controllers
Found actor-2: underlay
Found actor-3: Evolution controller
Found actor-4: CL automation framework
Found actor-5: CI/CD pipeline
Found actor-6: network instance
Found actor-7: evolvable controllers
Found actor-8: Evolution as a service


### FG-AN-USECASE-027

In [111]:
app.create_actors_relationship_with_usecase("AN intent", "has", "scenarios for experimentation", "usecase_027")
app.create_actors_relationship_with_usecase("Experimentation controller", "import", "AN intent", "usecase_027")
app.create_actors_relationship_with_usecase("Experimentation controller", "export", "AN intent", "usecase_027")
app.create_actors_relationship_with_usecase("Experimentation controller", "trigger", "scenarios for experimentation", "usecase_027")
app.create_actors_relationship_with_usecase("Experimentation controller", "validate", "experimentation results", "usecase_027")
app.create_actors_relationship_with_usecase("Knowledge Base", "stores", "experimentation results", "usecase_027")
app.create_actors_relationship_with_usecase("Knowledge Base", "stores", "experimentation results", "usecase_027")
app.create_actors_relationship_with_usecase("underlay", "uses", "Experimentation as a service", "usecase_027")

app.create_actors_relationship_with_usecase("Experimentation as a service", "uses", "Experimentation controller", "usecase_027")
app.create_actors_relationship_with_usecase("Experimentation controller", "uses", "digital twin", "usecase_027")
app.create_actors_relationship_with_usecase("digital twin", "is", "underlay", "usecase_027")
app.create_actors_relationship_with_usecase("underlay specific configurations", "trigger", "Experimentation as a service", "usecase_027")
app.create_actors_relationship_with_usecase("Evolution controller", "trigger", "Experimentation as a service", "usecase_027")


Node already exists: AN intent in ['usecase_009_cat1', 'usecase_010_cat1', 'usecase_016', 'usecase_018']
adding label usecase_027 in addition to ['usecase_009_cat1', 'usecase_010_cat1', 'usecase_016', 'usecase_018']
Node already exists: scenarios for experimentation in ['usecase_002']
adding label usecase_027 in addition to ['usecase_002']
Created relation: AN intent - has - scenarios for experimentation in usecase_027
Node already exists: Experimentation controller in ['usecase_002', 'usecase_010_cat1', 'usecase_021']
adding label usecase_027 in addition to ['usecase_002', 'usecase_010_cat1', 'usecase_021']
Node already exists: AN intent in ['usecase_009_cat1', 'usecase_010_cat1', 'usecase_016', 'usecase_018', 'usecase_027']
ignoring ...
Created relation: Experimentation controller - import - AN intent in usecase_027
Node already exists: Experimentation controller in ['usecase_002', 'usecase_010_cat1', 'usecase_021', 'usecase_027']
ignoring ...
Node already exists: AN intent in ['usec

In [112]:
app.find_all_usecase_actors_label("usecase_027")

Found actor-1: Experimentation controller
Found actor-2: scenarios for experimentation
Found actor-3: underlay
Found actor-4: Evolution controller
Found actor-5: AN intent
Found actor-6: underlay specific configurations
Found actor-7: experimentation results
Found actor-8: Experimentation as a service
Found actor-9: digital twin
Found actor-10: Knowledge Base


### FG-AN-USECASE-028

In [113]:
app.create_actors_relationship_with_usecase("Evolution controller", "select", "reference points", "usecase_028")
app.create_actors_relationship_with_usecase("reference points", "uses", "data-driven neural networks", "usecase_028")
app.create_actors_relationship_with_usecase("reference points", "uses", "signalling", "usecase_028")

app.create_actors_relationship_with_usecase("Experiments", "uses", "reference points", "usecase_028")
app.create_actors_relationship_with_usecase("Experiments", "are", "training", "usecase_028")

app.create_actors_relationship_with_usecase("ML model", "are", "data-driven neural networks", "usecase_028")
app.create_actors_relationship_with_usecase("AN sandbox", "use", "experimentation setup", "usecase_028")
app.create_actors_relationship_with_usecase("AN sandbox", "use", "ML model", "usecase_028")
app.create_actors_relationship_with_usecase("simulators", "are", "SRC", "usecase_028")
app.create_actors_relationship_with_usecase("simulators", "are", "SINK", "usecase_028")
app.create_actors_relationship_with_usecase("underlay", "has", "SINK", "usecase_028")

app.create_actors_relationship_with_usecase("controllers", "modify", "control flow", "usecase_028")
app.create_actors_relationship_with_usecase("controllers", "modify", "data flow", "usecase_028")
app.create_actors_relationship_with_usecase("underlay", "has", "data flow", "usecase_028")
app.create_actors_relationship_with_usecase("underlay", "has", "control flow", "usecase_028")
app.create_actors_relationship_with_usecase("training", "need", "cost", "usecase_028")
app.create_actors_relationship_with_usecase("data-driven neural networks", "reduce", "signalling", "usecase_028")

app.create_actors_relationship_with_usecase("5G NR beam management", "are", "controllers", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "codebook", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "codebook", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "need", "data collection", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "antennas", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "gains", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "location", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "AoA", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "beam sweep", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "beam measurement", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "beam decision", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "pilot-less estimation", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "coordination free", "usecase_028")
app.create_actors_relationship_with_usecase("5G NR beam management", "has", "inference", "usecase_028")

app.create_actors_relationship_with_usecase("SRC", "has", "Signal-to-Noise-Ratio (SNR) levels", "usecase_028")
app.create_actors_relationship_with_usecase("SRC", "has", "beam measurement", "usecase_028")

Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024', 'usecase_026', 'usecase_027']
adding label usecase_028 in addition to ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024', 'usecase_026', 'usecase_027']
Node already exists: reference points in ['usecase_010_cat1']
adding label usecase_028 in addition to ['usecase_010_cat1']
Created relation: Evolution controller - select - reference points in usecase_028
Node already exists: reference points in ['usecase_010_cat1', 'usecase_028']
ignoring ...
Created node: data-driven neural networks
Created relation: reference points - uses - data-driven neural networks in usecase_028
Node already exists: reference points in ['usecase_010_cat1

In [114]:
app.find_all_usecase_actors_label("usecase_028")

Found actor-1: controllers
Found actor-2: ML model
Found actor-3: AN sandbox
Found actor-4: simulators
Found actor-5: underlay
Found actor-6: Evolution controller
Found actor-7: inference
Found actor-8: reference points
Found actor-9: data collection
Found actor-10: SINK
Found actor-11: SRC
Found actor-12: training
Found actor-13: data-driven neural networks
Found actor-14: signalling
Found actor-15: Experiments
Found actor-16: experimentation setup
Found actor-17: control flow
Found actor-18: data flow
Found actor-19: cost
Found actor-20: 5G NR beam management
Found actor-21: codebook
Found actor-22: antennas
Found actor-23: gains
Found actor-24: location
Found actor-25: AoA
Found actor-26: beam sweep
Found actor-27: beam measurement
Found actor-28: beam decision
Found actor-29: pilot-less estimation
Found actor-30: coordination free
Found actor-31: Signal-to-Noise-Ratio (SNR) levels


### FG-AN-USECASE-029

In [115]:
app.create_actors_relationship_with_usecase("Evolution controller", "uses", 
                                        "controller repository", "usecase_029") 
app.create_actors_relationship_with_usecase("data collection", "need", 
                                        "evolution", "usecase_029")    
app.create_actors_relationship_with_usecase("modules", "need", 
                                        "evolution", "usecase_029")  
app.create_actors_relationship_with_usecase("Data analytics", "need", 
                                        "evolution", "usecase_029")   

app.create_actors_relationship_with_usecase("modules", "are", 
                                        "3rd party", 
                                        "usecase_029")          
app.create_actors_relationship_with_usecase("AR glasses", "are", 
                                        "data collection", 
                                        "usecase_029")    
app.create_actors_relationship_with_usecase("AR glasses", "has", 
                                        "QR code reading", 
                                        "usecase_029")  
app.create_actors_relationship_with_usecase("AR glasses", "has", 
                                        "optical character recognition", 
                                        "usecase_029") 
app.create_actors_relationship_with_usecase("AR glasses", "has", 
                                        "device port recognition", 
                                        "usecase_029")  
app.create_actors_relationship_with_usecase("AR glasses", "has", 
                                        "output rendering", 
                                        "usecase_029") 

app.create_actors_relationship_with_usecase("Intelligent Maintenance Assistance System", 
                                        "are", 
                                        "controllers", 
                                        "usecase_029")
app.create_actors_relationship_with_usecase("Intelligent Maintenance Assistance System", 
                                        "has", 
                                        "AR glasses", 
                                        "usecase_029")                                    
app.create_actors_relationship_with_usecase("Intelligent Maintenance Assistance System", 
                                        "has", 
                                        "ML model", 
                                        "usecase_029")   
app.create_actors_relationship_with_usecase("Intelligent Maintenance Assistance System", 
                                        "has", 
                                        "visual Simultaneous Localization And Mapping", 
                                        "usecase_029")   
app.create_actors_relationship_with_usecase("digital twin", 
                                        "has", 
                                        "visual Simultaneous Localization And Mapping", 
                                        "usecase_029")  
app.create_actors_relationship_with_usecase( 
                                        "visual Simultaneous Localization And Mapping", 
                                        "need",
                                        "real data",
                                        "usecase_029")
app.create_actors_relationship_with_usecase( 
                                        "visual Simultaneous Localization And Mapping", 
                                        "need",
                                        "simulated data",
                                        "usecase_029")
app.create_actors_relationship_with_usecase( 
                                        "simulators", 
                                        "generate",
                                        "simulated data",
                                        "usecase_029")
app.create_actors_relationship_with_usecase( 
                                        "underlay", 
                                        "generate",
                                        "real data",
                                        "usecase_029")
app.create_actors_relationship_with_usecase( 
                                        "Intelligent Maintenance Assistance System", 
                                        "generate",
                                        "reports",
                                        "usecase_029")
app.create_actors_relationship_with_usecase( 
                                        "Intelligent Maintenance Assistance System", 
                                        "integrate",
                                        "Software development kit",
                                        "usecase_029")
app.create_actors_relationship_with_usecase( 
                                        "3rd party", 
                                        "uses",
                                        "Software development kit",
                                        "usecase_029")


Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028']
adding label usecase_029 in addition to ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028']
Node already exists: controller repository in ['usecase_001', 'usecase_004']
adding label usecase_029 in addition to ['usecase_001', 'usecase_004']
Created relation: Evolution controller - uses - controller repository in usecase_029
Node already exists: data collection in ['usecase_013_cat2', 'usecase_017', 'usecase_028']
adding label usecase_029 in addition to ['usecase_013_cat2', 'usecase_017', 'usecase_028']
Node already exists: evolution in ['us

In [116]:
app.find_all_usecase_actors_label("usecase_029")

Found actor-1: controller repository
Found actor-2: controllers
Found actor-3: ML model
Found actor-4: reports
Found actor-5: simulators
Found actor-6: underlay
Found actor-7: Evolution controller
Found actor-8: Data analytics
Found actor-9: modules
Found actor-10: evolution
Found actor-11: data collection
Found actor-12: 3rd party
Found actor-13: digital twin
Found actor-14: AR glasses
Found actor-15: QR code reading
Found actor-16: optical character recognition
Found actor-17: device port recognition
Found actor-18: output rendering
Found actor-19: Intelligent Maintenance Assistance System
Found actor-20: visual Simultaneous Localization And Mapping
Found actor-21: real data
Found actor-22: simulated data
Found actor-23: Software development kit


### FG-AN-USECASE-030

In [117]:
app.create_actors_relationship_with_usecase( 
                                        "controllers", 
                                        "are",
                                        "3rd party",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "controller repository", 
                                        "are",
                                        "3rd party",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "evolution", 
                                        "uses",
                                        "underlay capabilities",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "underlay", 
                                        "publish",
                                        "underlay capabilities",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "controllers", 
                                        "has",
                                        "controller placement",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "controller placement", 
                                        "has",
                                        "placement algorithms",
                                        "usecase_030")

app.create_actors_relationship_with_usecase( 
                                        "edge network", 
                                        "provides",
                                        "data collection",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "data collection", 
                                        "has",
                                        "resource status",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "edge network", 
                                        "provides",
                                        "data collection",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "service migration methods", 
                                        "is",
                                        "controllers",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "edge network", 
                                        "has",
                                        "underlay specific APIs",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "underlay specific APIs", 
                                        "enable",
                                        "controller deployment",
                                        "usecase_030")
app.create_actors_relationship_with_usecase( 
                                        "service migration methods", 
                                        "has",
                                        "Data analytics",
                                        "usecase_030")

Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017', 'usecase_023', 'usecase_024', 'usecase_025', 'usecase_026', 'usecase_028', 'usecase_029']
adding label usecase_030 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017', 'usecase_023', 'usecase_024', 'usecase_025', 'usecase_026', 'usecase_028', 'usecase_029']
Node already exists: 3rd party in ['usecase_015', 'usecase_029']
adding label usecase_030 in addition to ['usecase_015', 'usecase_029']
Created relation: controllers - are - 3rd part

In [118]:
app.find_all_usecase_actors_label("usecase_030")

Found actor-1: controller repository
Found actor-2: controllers
Found actor-3: underlay
Found actor-4: Data analytics
Found actor-5: underlay specific APIs
Found actor-6: underlay capabilities
Found actor-7: controller deployment
Found actor-8: evolution
Found actor-9: data collection
Found actor-10: 3rd party
Found actor-11: controller placement
Found actor-12: edge network
Found actor-13: placement algorithms
Found actor-14: resource status
Found actor-15: service migration methods


### FG-AN-USECASE-031

In [119]:
app.create_actors_relationship_with_usecase( 
                                        "Experimentation controller", 
                                        "input",
                                        "evolvable controllers",
                                        "usecase_031")
app.create_actors_relationship_with_usecase( 
                                        "adaptation controller", 
                                        "input",
                                        "evolvable controllers",
                                        "usecase_031")
app.create_actors_relationship_with_usecase( 
                                        "controller repository", 
                                        "stores",
                                        "evolvable controllers",
                                        "usecase_031")
app.create_actors_relationship_with_usecase( 
                                        "intent", 
                                        "uses",
                                        "standard format",
                                        "usecase_031")


Node already exists: Experimentation controller in ['usecase_002', 'usecase_010_cat1', 'usecase_021', 'usecase_027']
adding label usecase_031 in addition to ['usecase_002', 'usecase_010_cat1', 'usecase_021', 'usecase_027']
Node already exists: evolvable controllers in ['usecase_026']
adding label usecase_031 in addition to ['usecase_026']
Created relation: Experimentation controller - input - evolvable controllers in usecase_031
Created node: adaptation controller
Node already exists: evolvable controllers in ['usecase_026', 'usecase_031']
ignoring ...
Created relation: adaptation controller - input - evolvable controllers in usecase_031
Node already exists: controller repository in ['usecase_001', 'usecase_004', 'usecase_029', 'usecase_030']
adding label usecase_031 in addition to ['usecase_001', 'usecase_004', 'usecase_029', 'usecase_030']
Node already exists: evolvable controllers in ['usecase_026', 'usecase_031']
ignoring ...
Created relation: controller repository - stores - evolv

In [120]:
app.find_all_usecase_actors_label("usecase_031")

Found actor-1: controller repository
Found actor-2: Experimentation controller
Found actor-3: intent
Found actor-4: evolvable controllers
Found actor-5: adaptation controller
Found actor-6: standard format


### FG-AN-USECASE-032

In [121]:
app.create_actors_relationship_with_usecase( 
                                        "Evolution controller", 
                                        "design",
                                        "inter-controller interface",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "3rd party", 
                                        "implements",
                                        "inter-controller interface",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "inter-controller interface", 
                                        "has",
                                        "interaction strategies",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "strategies", 
                                        "are",
                                        "interaction strategies",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "game theory", 
                                        "are",
                                        "strategies",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "strategies", 
                                        "need",
                                        "monitoring",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "strategies", 
                                        "need",
                                        "optimization",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "strategies", 
                                        "need",
                                        "experiments",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "resource allocation", 
                                        "is",
                                        "controllers",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "small cells", 
                                        "need",
                                        "resource allocation",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "resource allocation", 
                                        "need",
                                        "subchannel allocation",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "resource allocation", 
                                        "need",
                                        "power allocation",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "resource allocation", 
                                        "need",
                                        "power allocation",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "game theory", 
                                        "has",
                                        "players",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "players", 
                                        "need",
                                        "evolution",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "resource allocation", 
                                        "has",
                                        "outer loop",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "outer loop", 
                                        "has",
                                        "equilibrium inference",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "resource allocation", 
                                        "has",
                                        "inner loop",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "inner loop", 
                                        "has",
                                        "resource optimization",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "strategies", 
                                        "has",
                                        "tradeoff",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "tradeoff", 
                                        "is",
                                        "gain",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "tradeoff", 
                                        "is",
                                        "low latency",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "tradeoff", 
                                        "is",
                                        "privacy",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "players", 
                                        "is",
                                        "cooperating",
                                        "usecase_032")
app.create_actors_relationship_with_usecase( 
                                        "players", 
                                        "is",
                                        "non-cooperating",
                                        "usecase_032")

Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028', 'usecase_029']
adding label usecase_032 in addition to ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028', 'usecase_029']
Created node: inter-controller interface
Created relation: Evolution controller - design - inter-controller interface in usecase_032
Node already exists: 3rd party in ['usecase_015', 'usecase_029', 'usecase_030']
adding label usecase_032 in addition to ['usecase_015', 'usecase_029', 'usecase_030']
Node already exists: inter-controller interface in ['usecase_032']
ignoring ...
Created relation: 3rd party - implements - i

In [122]:
app.find_all_usecase_actors_label("usecase_032")

Found actor-1: controllers
Found actor-2: experiments
Found actor-3: Evolution controller
Found actor-4: optimization
Found actor-5: evolution
Found actor-6: monitoring
Found actor-7: 3rd party
Found actor-8: outer loop
Found actor-9: inner loop
Found actor-10: tradeoff
Found actor-11: low latency
Found actor-12: inter-controller interface
Found actor-13: interaction strategies
Found actor-14: strategies
Found actor-15: game theory
Found actor-16: resource allocation
Found actor-17: small cells
Found actor-18: subchannel allocation
Found actor-19: power allocation
Found actor-20: players
Found actor-21: equilibrium inference
Found actor-22: resource optimization
Found actor-23: gain
Found actor-24: privacy
Found actor-25: cooperating
Found actor-26: non-cooperating


### FG-AN-USECASE-033

In [123]:
app.create_actors_relationship_with_usecase( 
                                        "Adaptation controller", 
                                        "has",
                                        "customization",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "controllers", 
                                        "need",
                                        "customization",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "customization", 
                                        "are",
                                        "runtime",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "customization", 
                                        "are",
                                        "underlay specific configurations",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "tools", 
                                        "has",
                                        "visualization",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "tools", 
                                        "has",
                                        "development environment",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "controllers", 
                                        "need",
                                        "tools",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "Evolution controller", 
                                        "integrates",
                                        "3rd party",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "service automation", 
                                        "is",
                                        "controllers",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "service automation", 
                                        "has",
                                        "workflows",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "workflows", 
                                        "has",
                                        "tasks",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "workflows", 
                                        "has",
                                        "definitions",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "definitions", 
                                        "use",
                                        "domain specific language",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "tasks", 
                                        "are",
                                        "control tasks",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "tasks", 
                                        "are",
                                        "application tasks",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "workflows", 
                                        "are",
                                        "preloaded",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "workflows", 
                                        "need",
                                        "composed",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "workflows", 
                                        "need",
                                        "deployment",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "workflows", 
                                        "need",
                                        "validation",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "workflows", 
                                        "need",
                                        "monitoring",
                                        "usecase_033")
app.create_actors_relationship_with_usecase( 
                                        "workflows", 
                                        "need",
                                        "storage",
                                        "usecase_033")

Node already exists: Adaptation controller in ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_019', 'usecase_025']
adding label usecase_033 in addition to ['usecase_004', 'usecase_005', 'usecase_007_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_019', 'usecase_025']
Node already exists: customization in ['usecase_024']
adding label usecase_033 in addition to ['usecase_024']
Created relation: Adaptation controller - has - customization in usecase_033
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017', 'usecase_023', 'us

In [124]:
app.find_all_usecase_actors_label("usecase_033")

Found actor-1: controllers
Found actor-2: Adaptation controller
Found actor-3: Evolution controller
Found actor-4: validation
Found actor-5: monitoring
Found actor-6: 3rd party
Found actor-7: underlay specific configurations
Found actor-8: runtime
Found actor-9: customization
Found actor-10: tools
Found actor-11: visualization
Found actor-12: development environment
Found actor-13: service automation
Found actor-14: workflows
Found actor-15: tasks
Found actor-16: definitions
Found actor-17: domain specific language
Found actor-18: control tasks
Found actor-19: application tasks
Found actor-20: preloaded
Found actor-21: composed
Found actor-22: deployment
Found actor-23: storage


### FG-AN-USECASE-034

In [125]:
app.create_actors_relationship_with_usecase( 
                                        "AN orchestrator", 
                                        "need",
                                        "intent",
                                        "usecase_034")
app.create_actors_relationship_with_usecase( 
                                        "AN orchestrator", 
                                        "trigger",
                                        "Evolution controller",
                                        "usecase_034")
app.create_actors_relationship_with_usecase( 
                                        "Evolution controller", 
                                        "optimizes",
                                        "controller",
                                        "usecase_034")
app.create_actors_relationship_with_usecase( 
                                        "Adaptation controller", 
                                        "need",
                                        "controller",
                                        "usecase_034")
app.create_actors_relationship_with_usecase( 
                                        "Adaptation controller", 
                                        "need",
                                        "underlay specific configurations",
                                        "usecase_034")
app.create_actors_relationship_with_usecase( 
                                        "Adaptation controller", 
                                        "gives",
                                        "underlay integration",
                                        "usecase_034")
app.create_actors_relationship_with_usecase( 
                                        "Adaptation controller", 
                                        "monitor",
                                        "underlay integration",
                                        "usecase_034")
app.create_actors_relationship_with_usecase( 
                                        "Adaptation controller", 
                                        "has",
                                        "3rd party",
                                        "usecase_034")
app.create_actors_relationship_with_usecase( 
                                        "dataplane", 
                                        "is",
                                        "underlay",
                                        "usecase_034")
app.create_actors_relationship_with_usecase("dataplane", "has", "modules", "usecase_034")
app.create_actors_relationship_with_usecase("modules", "are", "Forward Error Correction", "usecase_034")
app.create_actors_relationship_with_usecase("modules", "are", "traffic compression", "usecase_034")
app.create_actors_relationship_with_usecase("modules", "are", "encapsulation", "usecase_034")
app.create_actors_relationship_with_usecase("modules", "are", "decapsulation", "usecase_034")
app.create_actors_relationship_with_usecase("dataplane", "need", "controllers", "usecase_034")
app.create_actors_relationship_with_usecase("Adaptation controller", "selects", "controllers", "usecase_034")
app.create_actors_relationship_with_usecase("Adaptation controller", "allocates", "controllers", "usecase_034")
app.create_actors_relationship_with_usecase("underlay", "deploy", "controllers", "usecase_034")
app.create_actors_relationship_with_usecase("underlay", "deploy", "controllers", "usecase_034")

app.create_actors_relationship_with_usecase("underlay orchestrator", "tightly coupled", "AN orchestrator", "usecase_034")
app.create_actors_relationship_with_usecase("underlay orchestrator", "loosely coupled", "AN orchestrator", "usecase_034")



Node already exists: AN orchestrator in ['usecase_010_cat1', 'usecase_011_cat1', 'usecase_016', 'usecase_018', 'usecase_025']
adding label usecase_034 in addition to ['usecase_010_cat1', 'usecase_011_cat1', 'usecase_016', 'usecase_018', 'usecase_025']
Node already exists: intent in ['usecase_005', 'usecase_009_cat2', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_016', 'usecase_018', 'usecase_031']
adding label usecase_034 in addition to ['usecase_005', 'usecase_009_cat2', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_016', 'usecase_018', 'usecase_031']
Created relation: AN orchestrator - need - intent in usecase_034
Node already exists: AN orchestrator in ['usecase_010_cat1', 'usecase_011_cat1', 'usecase_016', 'usecase_018', 'usecase_025', 'usecase_034']
ignoring ...
Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 

In [126]:
app.find_all_usecase_actors_label("usecase_034")

Found actor-1: controllers
Found actor-2: Adaptation controller
Found actor-3: underlay
Found actor-4: intent
Found actor-5: Evolution controller
Found actor-6: underlay orchestrator
Found actor-7: modules
Found actor-8: AN orchestrator
Found actor-9: 3rd party
Found actor-10: controller
Found actor-11: underlay specific configurations
Found actor-12: underlay integration
Found actor-13: dataplane
Found actor-14: Forward Error Correction
Found actor-15: traffic compression
Found actor-16: encapsulation
Found actor-17: decapsulation


### FG-AN-USECASE-035

In [127]:
app.create_actors_relationship_with_usecase("inner loop", "has", "sub-intent", "usecase_035")
app.create_actors_relationship_with_usecase("inner loop", "has", "domain specific aspects", "usecase_035")
app.create_actors_relationship_with_usecase("Evolution controller", "optimizes", "sub-intent", "usecase_035")
app.create_actors_relationship_with_usecase("Evolution controller", "optimizes", "sub-intent", "usecase_035")
app.create_actors_relationship_with_usecase("simulators", "have", "domain specific aspects", "usecase_035")

app.create_actors_relationship_with_usecase("AutoML", "is", "underlay", "usecase_035")
app.create_actors_relationship_with_usecase("AutoML controller", "is", "controllers", "usecase_035")
app.create_actors_relationship_with_usecase("AutoML controller", "finds", "ML model optimization", "usecase_035")
app.create_actors_relationship_with_usecase("AN orchestrator", "selects", "underlay", "usecase_035")
app.create_actors_relationship_with_usecase("AutoML controller", "selects", "experiments", "usecase_035")
app.create_actors_relationship_with_usecase("AutoML controller", "optimizes", "hyperparameters", "usecase_035")





Node already exists: inner loop in ['usecase_018', 'usecase_032']
adding label usecase_035 in addition to ['usecase_018', 'usecase_032']
Node already exists: sub-intent in ['usecase_018']
adding label usecase_035 in addition to ['usecase_018']
Created relation: inner loop - has - sub-intent in usecase_035
Node already exists: inner loop in ['usecase_018', 'usecase_032', 'usecase_035']
ignoring ...
Node already exists: domain specific aspects in ['usecase_022']
adding label usecase_035 in addition to ['usecase_022']
Created relation: inner loop - has - domain specific aspects in usecase_035
Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028', 'usecase_029', 'usecase_032', 'usecase_033', 'usecase_034']
adding label usecase_035 in addition to ['usecase_006_cat1', 'useca

In [128]:
app.find_all_usecase_actors_label("usecase_035")

Found actor-1: controllers
Found actor-2: simulators
Found actor-3: experiments
Found actor-4: underlay
Found actor-5: Evolution controller
Found actor-6: AN orchestrator
Found actor-7: sub-intent
Found actor-8: inner loop
Found actor-9: domain specific aspects
Found actor-10: AutoML
Found actor-11: AutoML controller
Found actor-12: ML model optimization
Found actor-13: hyperparameters


### FG-AN-USECASE-036

In [129]:
app.create_actors_relationship_with_usecase("RAN slicing controller", "is", "controllers", "usecase_036")
app.create_actors_relationship_with_usecase("VNF placement controller", "is", "controllers", "usecase_036")
app.create_actors_relationship_with_usecase("SDN controller", "is", "SRC", "usecase_036")
app.create_actors_relationship_with_usecase("RAN controller", "is", "SRC", "usecase_036")

app.create_actors_relationship_with_usecase("SDN controller", "is", "SINK", "usecase_036")
app.create_actors_relationship_with_usecase("RAN controller", "is", "SINK", "usecase_036")

app.create_actors_relationship_with_usecase("controllers", "use", "AI agent", "usecase_036")

app.create_actors_relationship_with_usecase("AI agent", "use", "ML model", "usecase_036")
app.create_actors_relationship_with_usecase("AI agent", "select", "ML model", "usecase_036")
app.create_actors_relationship_with_usecase("AI agent", "optimize", "ML model", "usecase_036")
app.create_actors_relationship_with_usecase("AI agent", "tightly coupled", "AN orchestrator", "usecase_036")
app.create_actors_relationship_with_usecase("AI agent", "loosely coupled", "AN orchestrator", "usecase_036")

Created node: RAN slicing controller
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017', 'usecase_023', 'usecase_024', 'usecase_025', 'usecase_026', 'usecase_028', 'usecase_029', 'usecase_030', 'usecase_032', 'usecase_033', 'usecase_034', 'usecase_035']
adding label usecase_036 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017', 'usecase_023', 'usecase_024', 'usecase_025', 'usecase_026', 'usecase_028', 'usecase_029', 'usecase_030', 'usecase_032', 'usecase_033', 'usecase_034', 'usecase_0

In [130]:
app.find_all_usecase_actors_label("usecase_036")

Found actor-1: controllers
Found actor-2: ML model
Found actor-3: AN orchestrator
Found actor-4: SINK
Found actor-5: SRC
Found actor-6: RAN slicing controller
Found actor-7: VNF placement controller
Found actor-8: SDN controller
Found actor-9: RAN controller
Found actor-10: AI agent


### FG-AN-USECASE-037

In [131]:
app.create_actors_relationship_with_usecase("network sharing controller", "is", "controllers", "usecase_037")
app.create_actors_relationship_with_usecase("network sharing controller", "has", "resource scaling", "usecase_037")

app.create_actors_relationship_with_usecase("resource allocation", "across", "network operator domains", "usecase_037")
app.create_actors_relationship_with_usecase("resource allocation", "has", "dynamic service agreement", "usecase_037")

app.create_actors_relationship_with_usecase("network operator domains", "are", "RAN", "usecase_037")
app.create_actors_relationship_with_usecase("network operator domains", "are", "transport network", "usecase_037")
app.create_actors_relationship_with_usecase("network operator domains", "are", "Core network", "usecase_037")

app.create_actors_relationship_with_usecase("dynamic service agreement", "are", "smart contract", "usecase_037")

Created node: network sharing controller
Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017', 'usecase_023', 'usecase_024', 'usecase_025', 'usecase_026', 'usecase_028', 'usecase_029', 'usecase_030', 'usecase_032', 'usecase_033', 'usecase_034', 'usecase_035', 'usecase_036']
adding label usecase_037 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017', 'usecase_023', 'usecase_024', 'usecase_025', 'usecase_026', 'usecase_028', 'usecase_029', 'usecase_030', 'usecase_032', 'usecase_033', 'useca

In [132]:
app.find_all_usecase_actors_label("usecase_037")

Found actor-1: controllers
Found actor-2: resource allocation
Found actor-3: network sharing controller
Found actor-4: resource scaling
Found actor-5: network operator domains
Found actor-6: dynamic service agreement
Found actor-7: RAN
Found actor-8: transport network
Found actor-9: Core network
Found actor-10: smart contract


### FG-AN-USECASE-038

In [133]:
app.create_actors_relationship_with_usecase("supply chain management controller", "is", "controller", "usecase_038")
app.create_actors_relationship_with_usecase("supply chain management controller", "monitors", "underlay", "usecase_038")
app.create_actors_relationship_with_usecase("supply chain management controller", "finds", "failure", "usecase_038")
app.create_actors_relationship_with_usecase("supply chain management controller", "raises", "purchase order", "usecase_038")
app.create_actors_relationship_with_usecase("supply chain management controller", "notify", "NF configure request", "usecase_038")
app.create_actors_relationship_with_usecase("NF configure request", "needs", "underlay specific configurations", "usecase_038")
app.create_actors_relationship_with_usecase("supply chain management controller", "triggers", "Experimentation controller", "usecase_038")
app.create_actors_relationship_with_usecase("supply chain management controller", "triggers", "Adaptation controller", "usecase_038")

Created node: supply chain management controller
Node already exists: controller in ['usecase_018', 'usecase_034']
adding label usecase_038 in addition to ['usecase_018', 'usecase_034']
Created relation: supply chain management controller - is - controller in usecase_038
Node already exists: supply chain management controller in ['usecase_038']
ignoring ...
Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019', 'usecase_020', 'usecase_021', 'usecase_023', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028', 'usecase_029', 'usecase_030', 'usecase_034', 'usecase_035']
adding label usecase_038 in addition to ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019', 'usecase_020', 'usecase_021', 'usecase_023', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028', 'usecase_029', 'usecase_030', 'usecase_034', 'usecase_035']
Created relation: supply chain mana

In [134]:
app.find_all_usecase_actors_label("usecase_038")

Found actor-1: Experimentation controller
Found actor-2: Adaptation controller
Found actor-3: underlay
Found actor-4: controller
Found actor-5: underlay specific configurations
Found actor-6: failure
Found actor-7: supply chain management controller
Found actor-8: purchase order
Found actor-9: NF configure request


### FG-AN-USECASE-039

In [135]:
app.create_actors_relationship_with_usecase("Evolution controller", "discovers", 
                                        "use cases", "usecase_039") 
app.create_actors_relationship_with_usecase("Evolution controller", "search", 
                                        "controller repository", "usecase_039") 
app.create_actors_relationship_with_usecase("Evolution controller", "finds", 
                                        "controllers", "usecase_039") 
app.create_actors_relationship_with_usecase("Evolution controller", "triggers", 
                                        "Experimentation controller", "usecase_039") 
app.create_actors_relationship_with_usecase("Evolution controller", "needs", 
                                        "controller selection", "usecase_039") 
app.create_actors_relationship_with_usecase("Adaptation controller", "providers", 
                                        "controller selection", "usecase_039") 
app.create_actors_relationship_with_usecase("Adaptation controller", "providers", 
                                        "monitoring", "usecase_039") 
app.create_actors_relationship_with_usecase("Adaptation controller", "providers", 
                                        "reports", "usecase_039") 

Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028', 'usecase_029', 'usecase_032', 'usecase_033', 'usecase_034', 'usecase_035']
adding label usecase_039 in addition to ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028', 'usecase_029', 'usecase_032', 'usecase_033', 'usecase_034', 'usecase_035']
Node already exists: use cases in ['usecase_006_cat1', 'usecase_008_cat1']
adding label usecase_039 in addition to ['usecase_006_cat1', 'usecase_008_cat1']
Created relation: Evolution controller - discovers - use cases in usecase_039
Node already exists: Evolution controller in ['usecase_006_cat1', 'us

In [136]:
app.find_all_usecase_actors_label("usecase_039")

Found actor-1: controller repository
Found actor-2: controllers
Found actor-3: reports
Found actor-4: Experimentation controller
Found actor-5: Adaptation controller
Found actor-6: Evolution controller
Found actor-7: use cases
Found actor-8: monitoring
Found actor-9: controller selection


### FG-AN-USECASE-040

In [137]:
app.create_actors_relationship_with_usecase("traffic offload controller", "is", "controller", "usecase_040") 
app.create_actors_relationship_with_usecase("traffic offload controller", "monitor", "underlay", "usecase_040") 
app.create_actors_relationship_with_usecase("traffic offload controller", "trigger", "traffic offload request", "usecase_040") 
app.create_actors_relationship_with_usecase("traffic offload controller", "evaluate", "SLA response", "usecase_040") 
app.create_actors_relationship_with_usecase("traffic offload controller", "triggers", "Adaptation controller", "usecase_040") 
app.create_actors_relationship_with_usecase("traffic offload controller", "configures", "traffic offload", "usecase_040") 
app.create_actors_relationship_with_usecase("traffic offload controller", "discover", "traffic offload controller", "usecase_040") 

Created node: traffic offload controller
Node already exists: controller in ['usecase_018', 'usecase_034', 'usecase_038']
adding label usecase_040 in addition to ['usecase_018', 'usecase_034', 'usecase_038']
Created relation: traffic offload controller - is - controller in usecase_040
Node already exists: traffic offload controller in ['usecase_040']
ignoring ...
Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019', 'usecase_020', 'usecase_021', 'usecase_023', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028', 'usecase_029', 'usecase_030', 'usecase_034', 'usecase_035', 'usecase_038']
adding label usecase_040 in addition to ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_017', 'usecase_019', 'usecase_020', 'usecase_021', 'usecase_023', 'usecase_024', 'usecase_026', 'usecase_027', 'usecase_028', 'usecase_029', 'usecase_030', 'usecase_034', 'usecase_035', 'usecase_038']

In [138]:
app.find_all_usecase_actors_label("usecase_040")

Found actor-1: Adaptation controller
Found actor-2: underlay
Found actor-3: controller
Found actor-4: traffic offload controller
Found actor-5: traffic offload request
Found actor-6: SLA response
Found actor-7: traffic offload


## Final Step: close the database connection

In [139]:
app.close()